# <u><b>Final Project: Student Loan Dataset</b></u>
## Milestone 3
## By Paulina Mostek
## <u>Objective:</u> Perform data cleaning using pandas on the National Student Loan Data System dataset using Python.
### Datasets used: https://catalog.data.gov/dataset/national-student-loan-data-system-722b0
* 1617FedSchoolCodeList.xlsx (Federal School Code List) - This may be needed to crossreference school data
* FL_Dashboard_AY_2009_2010_Q1.xls (Q1 of 2009-2010 School Year)
### We will be exploring the original Excel files, loading them into pandas dataframes, and perform data cleaning.

# <u>Import libraries</u>
## The following libraries will be imported for our code:
* csv: For reading xls files as csv.
* pandas: For loading data into dataframes.
* numpy: For handling arrays.
* matplotlib and seaborns: For plotting data observations.
* statistics: For exploratory analysis and checking basic information of the table.
* xlrd: This library must be installed (pip install xlrd) AND imported to read XLS files.
* re: For using regular expressions (regex) for string manipulation.

In [2]:
import csv
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statistics as stats
import seaborn as sns
import xlrd
import re

# Manually import sheets: Import each sheet from the original Excel file as its own pandas dataframe
### Load in....
* Q1: Quarterly Activity (QA)
* Q1: Award Year Summary (AYS)
* Federal School Code List (FSL)
#### We will use the .read_excel command because these files exist as sheets in an Excel file

In [3]:
qa = pd.read_excel("FL_Dashboard_AY2009_2010_Q1.xls", sheet_name="Quarterly Activity")
ays = pd.read_excel("FL_Dashboard_AY2009_2010_Q1.xls", sheet_name="Award Year Summary")
fsl = pd.read_excel("1617fedschoolcodelist.xls")

C:\Users\pmost\anaconda3\envs\first\Lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


# <u>Data Exploration: Questions</u>
### We will be asking the following questions of the data and plotting the results in a later module:
1. Do community colleges or technical colleges originate more loans overall?
2. Which states have the highest total loan disbursements?
3. Do private or public schools disburse more in federal student loans?
4. Do institutions with “college” in the name receive more loans than those with “university”?
5. Are unsubsidized loans more common at private colleges than public ones?
### For relevancy, we will only explore data concerning schools in the US. Foreign schools will be excluded from our analysis.

# <u>Data Cleaning</u>
## Identifying Issues
### Possible quality check issues in original file:
* Additional school types - "PROPRIETARY" may require deletion if containing empty values or comparing private vs public schools.
* Additional school types - "FOREIGN PUBLIC, FOREIGN PRIVATE" are displayed throughout the data, which need to be deleted as we are only investigating US schools.
* Missing rows for school code and financial information: values are "-" or "0" instead of numbers.
* Other columns may need to be investigated for missing rows.
* Several Rows in the "Award Year Summary" Table are missing, particularly for parent and grad loans.
* Puerto Rico is treated as a state, with state abbreviation "PR" and not listed as its own country or province, unlike foreign schools.

### File-specific issues:
#### Quarterly Analysis and Award Year Summary
* Missing values as well as "0s" will need to be handled during data cleaning
* The first few missing rows will need to be handled (dropped)
* Columns will need to be renamed to proper original column names (School, State, etc.)
* Column headers (FFEL SUBSIDIZED, FFEL PARENT PLUS) will need to be reintegrated into the data. They currently exist among empty rows.

#### Federal School List (School Codes)
* Each entry has a unique school code, address, and zip code, suggesting that there is not one address or school code for each school, but each school has different identifiable information for each of its campuses.
* School Code may identify each individual campus of a school, and not each individual school.
* Example: 5610 and 5611 do not refer to different schools, but instead different campuses of the same school, ITT.
* Schools may need to be grouped by name instead of state, as one school contains several school codes across several states.

## <u>Asking Questions</u>
### 1. Did your exploratory analysis from last week highlight anything that needs to be cleaned?
##### Answer: Yes. Leading rows, column names, and headers should be cleaned. Foreign schools should be removed, as well as schools from Puerto Rico (PR) as we are investigating US schools.

### 2. Investigate any NaN's that you find. Should these be replaced with another value? If so, what would be the best value? Should that record be deleted instead?
##### Various NaN's found across the first few rows of data.
* The first 3 rows of two dataframes were entirely missing, containing only NaN values
* Missing values for numeric values were converted to 0s.
* The rows that are entirely missing should be deleted as the column names were esssentially pushed down to the 5th row.
* Missing values should not be replaced. They should be investigated. If the row contains only one zero, it is possible that the school is simply small and underfunded. If the row contains two or more zeroes, it is possible that the school gave an incomplete report to the Federal Loan Database, and the entire row should be deleted.

### 3.  Are there outliers to any of the data items? What could explain those outliers? Should those records be changed or deleted?
##### Yes. After converting loan-information columns to numeric types and using .describe() to extract descriptive statistics, values of 0 were observed.
* The reason for these outliers is that missing values were converted to "0"s when downloading the Excel file and loading it into a pandas dataframe.
* These records should be changed.
* Some schools may contain a zero in any value (such as disbursements of a loan) due to lack of funding or small number of students.
* For this reason, we will only delete records containing more than one zero. If there are two or more zeroes in a row, it is likely that these are missing values, not true zeroes.
* By removing rows containing two or more zeroes, we will be altering the descriptive statistics (such as average and minimum values) to match real, complete records instead of including incomplete records.

### 4. Are the column names not in a format that makes it easy to refer to?
##### Yes. Column names are "Unnamed: 1", "Unnamed: 2", instead of their proper titles.
* We must change the column names to their proper titles.
* Columns are also grouped under loan categories (FFEL SUBSIDIZED), which we must keep in mind.

### 5. Do you need to merge multiple datasets into one?
##### Each dataset shows different information (Quarterly Analysis, Award Year Summary, School Federal Code List), so we may not need to merge datasets.
* Quarterly Analysis shows the loan information for each campus throughout the quarter.
* Award Year Summary

### 6. Do you need to split up a column's data into multiple columns or simplify its values?
##### No. We may need to do the opposite and lump together columns, such as total funding, or federal loans (which cover several columns at once).
* Headers exist in the original dataset (FFEL SUBSIDIZED, FFEL UNSUBSIDIZED, etc.). We will need to manipulate the data to recreate these groupings.


## Display data for cleaning
* Quarterly analysis
* Award Year Summary
* School Code List

In [4]:
qa

,2009-2010 Award Year FFEL Volume by School,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20,Unnamed: 21,Unnamed: 22,Unnamed: 23,Unnamed: 24
0,Award Year Quarterly Activity (07/01/2009-09/...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Data Run: 4/5/2012,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,FFEL SUBSIDIZED,NaN,NaN,NaN,NaN,...,FFEL PARENT PLUS,NaN,NaN,NaN,NaN,FFEL GRAD PLUS,NaN,NaN,NaN,NaN
4,OPE ID,School,State,Zip Code,School Type,Recipients,# of Loans Originated,$ of Loans Originated,# of Disbursements,$ of Disbursements,...,Recipients,# of Loans Originated,$ of Loans Originated,# of Disbursements,$ of Disbursements,Recipients,# of Loans Originated,$ of Loans Originated,# of Disbursements,$ of Disbursements
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3820,00393200,UNIVERSITY OF WYOMING,WY,820713663,PUBLIC,2699,2735,12341711,2738,6819888,...,195,207,1830536,209,950682,58,65,740877,66,416236
3821,00393300,WESTERN WYOMING COMMUNITY COLLEGE,WY,829010428,PUBLIC,214,214,731883,215,359742,...,2,2,13300,2,6650,0,0,0,0,0
3822,00728900,CENTRAL WYOMING COLLEGE,WY,825012215,PUBLIC,149,154,414959,154,212783,...,7,7,34978,7,17492,0,0,0,0,0
3823,00915700,WYOTECH,WY,820729519,PROPRIETARY,1049,1099,2554580,1220,1498483,...,478,490,7567391,522,4033026,0,0,0,0,0


In [5]:
ays

,2009-2010 Award Year FFEL Volume by School,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20,Unnamed: 21,Unnamed: 22,Unnamed: 23,Unnamed: 24
0,Award Year Cumulative Activity through Quarter...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Data Run: 4/5/2012,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,FFEL SUBSIDIZED,NaN,NaN,NaN,NaN,...,FFEL PARENT PLUS,NaN,NaN,NaN,NaN,FFEL GRAD PLUS,NaN,NaN,NaN,NaN
4,OPE ID,School,State,Zip Code,School Type,Recipients,# of Loans Originated,$ of Loans Originated,# of Disbursements,$ of Disbursements,...,Recipients,# of Loans Originated,$ of Loans Originated,# of Disbursements,$ of Disbursements,Recipients,# of Loans Originated,$ of Loans Originated,# of Disbursements,$ of Disbursements
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3820,00393200,UNIVERSITY OF WYOMING,WY,820713663,PUBLIC,2699,2735,12341711,2738,6819888,...,195,207,1830536,209,950682,58,65,740877,66,416236
3821,00393300,WESTERN WYOMING COMMUNITY COLLEGE,WY,829010428,PUBLIC,214,214,731883,215,359742,...,2,2,13300,2,6650,0,0,0,0,0
3822,00728900,CENTRAL WYOMING COLLEGE,WY,825012215,PUBLIC,149,154,414959,154,212783,...,7,7,34978,7,17492,0,0,0,0,0
3823,00915700,WYOTECH,WY,820729519,PROPRIETARY,1049,1099,2554580,1220,1498483,...,478,490,7567391,522,4033026,0,0,0,0,0


In [6]:
fsl

,SchoolCode,SchoolName,Address,City,StateCode,ZipCode,Province,Country,PostalCode
0,B04724,WIDENER UNIV SCHOOL OF LAW - DE,4601 CONCORD PIKE/PO BOX 7474,WILMINGTON,DE,19803,NaN,NaN,NaN
1,B06171,CENTER FOR ADVANCED STUDIES OF PUER,BOX S-4467,SAN JUAN,PR,902,NaN,NaN,NaN
2,B06511,PENTECOSTAL THEOLOGICAL SEMINARY,PO BOX 3330,CLEVELAND,TN,37320,NaN,NaN,NaN
3,B07022,THE CHICAGO SCHOOL OF PROF PSYCHOLOGY,325 NORTH WELLS STREET,CHICAGO,IL,60610,NaN,NaN,NaN
4,B07624,NATIONAL COLLEGE OF NATURAL MEDICINE,049 SW PORTER,PORTLAND,OR,97201,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
6975,042517,HOPE COLLEGE OF ARTS AND SCIENCES,1200 SOUTH WEST 3RD STREET,POMPANO BEACH,FL,33069,NaN,NaN,NaN
6976,E40419,BEAUTY INSTITUTE SCHWARZKOPF PROFESSIONA,1411 RAILROAD AVENUE,BELLINGHAM,WA,98225,NaN,NaN,NaN
6977,042205,BUTTE COUNTY REGIONAL OCCUPATIONAL PROGR,2491 CARMICHAEL DRIVE,CHICO,CA,95928,NaN,NaN,NaN
6978,G42404,UNIVERSIDAD ANA G. MENDEZ - CAMPUS VIRTU,1552 AVENUE PONCE DE LEON,SAN JUAN,PR,926,NaN,NaN,NaN


## Identify Outliers
* We will identify outliers in the data to determine if those records should be changed or deleted.
* We will use .describe() to look at descriptive statistics for each column and identify outliers

In [12]:
qa.describe()

,2009-2010 Award Year FFEL Volume by School,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20,Unnamed: 21,Unnamed: 22,Unnamed: 23,Unnamed: 24
count,3824,3821,3821,3803,3821,3822,3821,3821,3821,3821,...,3822,3821,3821,3821,3821,3822,3821,3821,3821,3821
unique,3824,3553,56,3794,8,1395,1399,3411,1417,3442,...,436,457,2252,463,2281,283,292,992,294,989
top,Award Year Quarterly Activity (07/01/2009-09/...,ITT TECHNICAL INSTITUTE,FC,841150000,PRIVATE,1,1,0,1,0,...,0,0,0,0,0,0,0,0,0,0
freq,1,29,381,2,1360,160,158,114,143,114,...,1352,1352,1352,1352,1352,2784,2784,2784,2784,2784


### Outcome: The colors are not numeric, since they are objects, so we instead see the following statistics:
* count: number of non-null entries
* unique: number of distinct values
* top: most frequent value
* freq: frequency of the top value

# <u> Data Cleaning: Quarterly Analysis (QA)</u>
## <u>Data Cleaning Plan:</u> To Identify Outliers
1. Create copies of the data so that we can perform cleaning on it without error
2.  Use row 4 (index 4) as your actual header.
3. Discard row 0–3.
* Rows are either entirely empty (0-2) or mostly empty but contain loan type headers (3).

4. Capture the loan groupings (from row 3) and attach them to the appropriate columns if needed (optional).

5. Clean column names like "Unnamed: 5" and assign meaningful multi-level headers or annotate with groupings.

## 1. Create copies of the data
#### In case of error during the cleaning process, we will be able to restart the process by making a copy of the original data.
* Quarterly Analysis
* Award Year System
* Federal School List

In [4]:
qa_clean = qa.copy()
ays_clean = ays.copy()
fsl_clean = fsl.copy()

## 2. Extract and store groupings (row index 3) for loan types "FFEL SUBSIDIZED, FFEL UNSUBSIDIZED" etc.
* Use iloc[] to select the loan grouping row
* Display the grouping row, which should show types of loans

In [5]:
group_row = qa_clean.iloc[3]  #This is the loan grouping row (Ex:"FFEL SUBSIDIZED")

In [13]:
group_row

2009-2010 Award Year FFEL Volume by School                  NaN
Unnamed: 1                                                  NaN
Unnamed: 2                                                  NaN
Unnamed: 3                                                  NaN
Unnamed: 4                                                  NaN
Unnamed: 5                                      FFEL SUBSIDIZED
Unnamed: 6                                                  NaN
Unnamed: 7                                                  NaN
Unnamed: 8                                                  NaN
Unnamed: 9                                                  NaN
Unnamed: 10                                   FFEL UNSUBSIDIZED
Unnamed: 11                                                 NaN
Unnamed: 12                                                 NaN
Unnamed: 13                                                 NaN
Unnamed: 14                                                 NaN
Unnamed: 15                             

## 3. Extract actual column names (row index 4)
* Use .iloc[] to select the row that contains proper column names
* Display the proper column names into a row called column_row

In [6]:
column_row = qa_clean.iloc[4]  #This contains: OPE ID, School Name, etc.


In [15]:
column_row

2009-2010 Award Year FFEL Volume by School                   OPE ID
Unnamed: 1                                                   School
Unnamed: 2                                                    State
Unnamed: 3                                                 Zip Code
Unnamed: 4                                              School Type
Unnamed: 5                                               Recipients
Unnamed: 6                                    # of Loans Originated
Unnamed: 7                                    $ of Loans Originated
Unnamed: 8                                       # of Disbursements
Unnamed: 9                                       $ of Disbursements
Unnamed: 10                                              Recipients
Unnamed: 11                                   # of Loans Originated
Unnamed: 12                                   $ of Loans Originated
Unnamed: 13                                      # of Disbursements
Unnamed: 14                                     

## 4.  Combine the grouping and actual column names into a multi-level index
* Replace NaNs in the group row with forward fill (to fill in group names)
### This command (forward fill) was found in pandas documentation: https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.ffill.html 
#### This command is used so that we can see which columns relate to specific loan types. We will later flatten the data so we can extract data by loan type.
* Display the filled names. We can now see which columns cover which loan type.
* Combine group + columnn name row into a MultiIndex

In [7]:
#Replace missing values in the loan group row with loan group types using forward fill
group_row_filled = group_row.fillna(method='ffill')
#Forward fill was found in pandas documentation (pandas.pydata.org)


C:\Users\pmost\AppData\Local\Temp\ipykernel_25884\1641007500.py:2: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  group_row_filled = group_row.fillna(method='ffill')


In [17]:
group_row_filled

2009-2010 Award Year FFEL Volume by School                  NaN
Unnamed: 1                                                  NaN
Unnamed: 2                                                  NaN
Unnamed: 3                                                  NaN
Unnamed: 4                                                  NaN
Unnamed: 5                                      FFEL SUBSIDIZED
Unnamed: 6                                      FFEL SUBSIDIZED
Unnamed: 7                                      FFEL SUBSIDIZED
Unnamed: 8                                      FFEL SUBSIDIZED
Unnamed: 9                                      FFEL SUBSIDIZED
Unnamed: 10                                   FFEL UNSUBSIDIZED
Unnamed: 11                                   FFEL UNSUBSIDIZED
Unnamed: 12                                   FFEL UNSUBSIDIZED
Unnamed: 13                                   FFEL UNSUBSIDIZED
Unnamed: 14                                   FFEL UNSUBSIDIZED
Unnamed: 15                             

In [8]:
#Combine group and column row into a MultiIndex
qa_clean.columns = pd.MultiIndex.from_arrays([group_row_filled, column_row])

## 5. Examine the first 5 rows

In [9]:
qa_clean.head(5)

3                                                NaN                           \
4                                             OPE ID  School  State  Zip Code   
0  Award Year Quarterly Activity  (07/01/2009-09/...     NaN    NaN       NaN   
1                                 Data Run: 4/5/2012     NaN    NaN       NaN   
2                                                        NaN    NaN       NaN   
3                                                NaN     NaN    NaN       NaN   
4                                             OPE ID  School  State  Zip Code   

3               FFEL SUBSIDIZED                                                \
4  School Type       Recipients  # of Loans Originated  $ of Loans Originated   
0          NaN              NaN                    NaN                    NaN   
1          NaN              NaN                    NaN                    NaN   
2          NaN              NaN                    NaN                    NaN   
3          NaN  FFEL SUBSIDIZED                    NaN                    NaN   
4  School Type       Recipients  # of Loans Originated  $ of Loans Originated   

3                                          ...  FFEL PARENT PLUS  \
4  # of Disbursements  $ of Disbursements  ...        Recipients   
0                 NaN                 NaN  ...               NaN   
1                 NaN                 NaN  ...               NaN   
2                 NaN                 NaN  ...               NaN   
3                 NaN                 NaN  ...  FFEL PARENT PLUS   
4  # of Disbursements  $ of Disbursements  ...        Recipients   

3                                                                    \
4  # of Loans Originated  $ of Loans Originated  # of Disbursements   
0                    NaN                    NaN                 NaN   
1                    NaN                    NaN                 NaN   
2                    NaN                    NaN                 NaN   
3                    NaN                    NaN                 NaN   
4  # of Loans Originated  $ of Loans Originated  # of Disbursements   

3                      FFEL GRAD PLUS                         \
4  $ of Disbursements      Recipients  # of Loans Originated   
0                 NaN             NaN                    NaN   
1                 NaN             NaN                    NaN   
2                 NaN             NaN                    NaN   
3                 NaN  FFEL GRAD PLUS                    NaN   
4  $ of Disbursements      Recipients  # of Loans Originated   

3                                                                 
4  $ of Loans Originated  # of Disbursements  $ of Disbursements  
0                    NaN                 NaN                 NaN  
1                    NaN                 NaN                 NaN  
2                    NaN                 NaN                 NaN  
3                    NaN                 NaN                 NaN  
4  $ of Loans Originated  # of Disbursements  $ of Disbursements  

[5 rows x 25 columns]

### Note:
* Loan categories lie at the top of the headers (3)
* Proper column names lie in the bottom of the headers (4)
* Original rows containing missing values, loan categories, and column names are kept and will now be discarded.

## 6. Drop the first 5 rows (missing values + headers)
* Use .iloc[] and slicing (:) to extract the first 5 rows.
* Starting at the 5th row (where the actual column names are), we sill select the remaining rows and reset the index to reformat the dataframe.
* Reset the index to drop the old index structure isntead of adding it as a new column (maintains index integrity)
* Save to new dataframe <u>qa_clean2</u>
* Inspect the dataframe for proper columns

In [10]:
qa_clean2 = qa_clean.iloc[5:].reset_index(drop=True)


In [11]:
qa_clean2.head(5)

3       NaN                                                                \
4    OPE ID                          School State   Zip Code  School Type   
0  00106100       ALASKA PACIFIC UNIVERSITY    AK  995084672      PRIVATE   
1  00106300  UNIVERSITY OF ALASKA FAIRBANKS    AK  997757500       PUBLIC   
2  00106500  UNIVERSITY OF ALASKA SOUTHEAST    AK  998018680       PUBLIC   
3  01146200  UNIVERSITY OF ALASKA ANCHORAGE    AK  995088050       PUBLIC   
4  02541000           ALASKA CAREER COLLEGE    AK  995071033  PROPRIETARY   

3 FFEL SUBSIDIZED                                              \
4      Recipients # of Loans Originated $ of Loans Originated   
0             291                   291               1546994   
1            1413                  1434               6394735   
2             406                   409               1866473   
3            2998                  3042              12780036   
4              38                    38                103869   

3                                        ... FFEL PARENT PLUS  \
4 # of Disbursements $ of Disbursements  ...       Recipients   
0                292             830513  ...               31   
1               1455            3290699  ...               36   
2                439            1044946  ...                8   
3               3045            6440086  ...              123   
4                 38              52178  ...                0   

3                                                                 \
4 # of Loans Originated $ of Loans Originated # of Disbursements   
0                    33                386770                 35   
1                    40                265784                 40   
2                     8                 88046                  8   
3                   125               1288224                125   
4                     0                     0                  0   

3                    FFEL GRAD PLUS                        \
4 $ of Disbursements     Recipients # of Loans Originated   
0             192181              5                     5   
1             138160              4                     4   
2              44024              0                     0   
3             645683              9                    11   
4                  0              0                     0   

3                                                              
4 $ of Loans Originated # of Disbursements $ of Disbursements  
0                 69730                  5              34865  
1                 68620                  4              28988  
2                     0                  0                  0  
3                 43878                 11              24063  
4                     0                  0                  0  

[5 rows x 25 columns]

### Now the dataframe (qa_clean2) has multi-level column names:
* Level 0: Loan category (FFEL SUBSIDIZED)
* Level 1: Actual column names ("School", etc.)

#### Multi-index columns
* The columns appear as: ('FFEL SUBSIDIZED', 'Loan Volume')
* The columns can be accessed like so: qa_clean2[('FFEL SUBSIDIZED', 'Loan Volume')]


### Improvements:
* Proper column names
* Subcategory of loan types within column index
* Rows filled with missing values have been deleted
### Further cleaning is needed:
* Convert values (only the numbers) to numeric type so we can perform operations on them
* Delete rows containing foreign schools (school type contains the word "FOREIGN") as we will only be examining US schools
* Check for missing values ("0"s).
* Delete rows containing missing values, which are marked with "0"
* Create a copy of this dataframe where layers are "flattened", so the dataframe does not have multi-level column names. This will aid plotting later on as accessing specific columns will become easier, and we can simply group together the loan type columns by matching the column name to the loan type, such as "SUBSIDIZED."


## 7. Convert numbers to numeric (still works with multi-index)
* Only change the columns from index 5 onward, as columns 0-4 contain letters and should not be changed to numbers.
* Display the data
* Use .dtypes() to check that the columns have been converted to numeric types
* use .describe() on the general data to extract descriptive statistics

In [12]:
for col in qa_clean2.columns[5:]:
    qa_clean2[col] = pd.to_numeric(qa_clean2[col], errors='coerce')



In [28]:
qa_clean2

3          NaN                                                      \
4       OPE ID                             School State   Zip Code   
0     00106100          ALASKA PACIFIC UNIVERSITY    AK  995084672   
1     00106300     UNIVERSITY OF ALASKA FAIRBANKS    AK  997757500   
2     00106500     UNIVERSITY OF ALASKA SOUTHEAST    AK  998018680   
3     01146200     UNIVERSITY OF ALASKA ANCHORAGE    AK  995088050   
4     02541000              ALASKA CAREER COLLEGE    AK  995071033   
...        ...                                ...   ...        ...   
3815  00393200              UNIVERSITY OF WYOMING    WY  820713663   
3816  00393300  WESTERN WYOMING COMMUNITY COLLEGE    WY  829010428   
3817  00728900            CENTRAL WYOMING COLLEGE    WY  825012215   
3818  00915700                            WYOTECH    WY  820729519   
3819  00925900   LARAMIE COUNTY COMMUNITY COLLEGE    WY  820073299   

3                 FFEL SUBSIDIZED                                              \
4     School Type      Recipients # of Loans Originated $ of Loans Originated   
0         PRIVATE             291                   291               1546994   
1          PUBLIC            1413                  1434               6394735   
2          PUBLIC             406                   409               1866473   
3          PUBLIC            2998                  3042              12780036   
4     PROPRIETARY              38                    38                103869   
...           ...             ...                   ...                   ...   
3815       PUBLIC            2699                  2735              12341711   
3816       PUBLIC             214                   214                731883   
3817       PUBLIC             149                   154                414959   
3818  PROPRIETARY            1049                  1099               2554580   
3819       PUBLIC             281                   291                885794   

3                                           ... FFEL PARENT PLUS  \
4    # of Disbursements $ of Disbursements  ...       Recipients   
0                   292             830513  ...               31   
1                  1455            3290699  ...               36   
2                   439            1044946  ...                8   
3                  3045            6440086  ...              123   
4                    38              52178  ...                0   
...                 ...                ...  ...              ...   
3815               2738            6819888  ...              195   
3816                215             359742  ...                2   
3817                154             212783  ...                7   
3818               1220            1498483  ...              478   
3819                291             447794  ...                8   

3                                                                    \
4    # of Loans Originated $ of Loans Originated # of Disbursements   
0                       33                386770                 35   
1                       40                265784                 40   
2                        8                 88046                  8   
3                      125               1288224                125   
4                        0                     0                  0   
...                    ...                   ...                ...   
3815                   207               1830536                209   
3816                     2                 13300                  2   
3817                     7                 34978                  7   
3818                   490               7567391                522   
3819                     8                 41636                  8   

3                       FFEL GRAD PLUS                        \
4    $ of Disbursements     Recipients # of Loans Originated   
0                192181              5                     5   
1                138160           

#### Success: The column values appear to be numeric. Non-numeric columns (school code, school name, etc.) remain unchanged

In [31]:
qa_clean2.dtypes

3                  4                    
NaN                OPE ID                   object
                   School                   object
                   State                    object
                   Zip Code                 object
                   School Type              object
FFEL SUBSIDIZED    Recipients                int64
                   # of Loans Originated     int64
                   $ of Loans Originated     int64
                   # of Disbursements        int64
                   $ of Disbursements        int64
FFEL UNSUBSIDIZED  Recipients                int64
                   # of Loans Originated     int64
                   $ of Loans Originated     int64
                   # of Disbursements        int64
                   $ of Disbursements        int64
FFEL PARENT PLUS   Recipients                int64
                   # of Loans Originated     int64
                   $ of Loans Originated     int64
                   # of Disbursements    

##### Success: All column types, except for the first few, are integers. This means we can perform operations and extract descriptive statistics.

#### Use .describe() to extract descriptive statistics
* Use .describe() on the general data

In [29]:
qa_clean2.describe()

3     FFEL SUBSIDIZED                                              \
4          Recipients # of Loans Originated $ of Loans Originated   
count     3820.000000           3820.000000          3.820000e+03   
mean       735.766230            747.539267          3.404240e+06   
std       2022.902051           2060.049041          9.368663e+06   
min          0.000000              0.000000          0.000000e+00   
25%         18.000000             18.000000          7.229875e+04   
50%        159.000000            160.000000          6.665555e+05   
75%        720.500000            728.250000          3.062910e+06   
max      67590.000000          68099.000000          2.833224e+08   

3                                           FFEL UNSUBSIDIZED  \
4     # of Disbursements $ of Disbursements        Recipients   
count        3820.000000       3.820000e+03       3820.000000   
mean          763.767539       1.708575e+06        713.747382   
std          2116.026539       4.865226e+06       2010.022622   
min             0.000000       0.000000e+00          0.000000   
25%            19.000000       3.585050e+04         15.000000   
50%           166.000000       3.215795e+05        145.000000   
75%           742.000000       1.521889e+06        688.250000   
max         70312.000000       1.548300e+08      62730.000000   

3                                                                     \
4     # of Loans Originated $ of Loans Originated # of Disbursements   
count           3820.000000          3.820000e+03        3820.000000   
mean             762.576440          4.215786e+06         779.637696   
std             2135.076524          1.269787e+07        2192.978381   
min                0.000000          0.000000e+00           0.000000   
25%               15.000000          7.457225e+04          16.000000   
50%              152.000000          6.948935e+05         156.500000   
75%              736.250000          3.454190e+06         757.500000   
max            63556.000000          3.750060e+08       65830.000000   

3                        FFEL PARENT PLUS                        \
4     $ of Disbursements       Recipients # of Loans Originated   
count       3.820000e+03      3820.000000           3820.000000   
mean        2.095755e+06        64.417801             66.702880   
std         6.514066e+06       183.029608            190.040276   
min         0.000000e+00         0.000000              0.000000   
25%         3.663150e+04         0.000000              0.000000   
50%         3.317690e+05         3.000000              3.000000   
75%         1.668416e+06        47.000000             48.000000   
max         1.985650e+08      3272.000000           3339.000000   

3                                                                  \
4     $ of Loans Originated # of Disbursements $ of Disbursements   
count          3.820000e+03        3820.000000       3.820000e+03   
mean           8.415002e+05          68.390052       4.273558e+05   
std            2.846121e+06         193.765012       1.446481e+06   
min            0.000000e+00           0.000000       0.000000e+00   
25%            0.000000e+00           0.000000       0.000000e+00   
50%            2.454050e+04           3.000000       1.179050e+04   
75%            4.573030e+05          48.000000       2.236740e+05   
max            8.802681e+07        3354.000000       4.467873e+07   

3     FFEL GRAD PLUS                                              \
4         Recipients # of Loans Originated $ of Loans Originated   
count    3820.000000           3820.000000          3.820000e+03   
mean       33.791361             36.476963          7.116602e+05   
std       175.446114            190.142882          4.493335e+06   
min         0.000000              0.000000          0.000000e+00   
25%         0.000000              0.000000          0.000000e+00   
50%         0.000000              0.000000          0.000000e+00   
75%         1.000000           

#### Although the decimals appear long, this is normal behavior for the .describe() command.
* .describe() Shows high-precision float values (decimals).
* If needed, these values can be adjusted when plotting to aid visibility in the display. For now, they can be left as-is.

In [38]:
qa_clean2.sample(5)

3          NaN                                                     \
4       OPE ID                            School State   Zip Code   
3742  00389700  UNIVERSITY OF WISCONSIN COLLEGES    WI  537152635   
1596  00200200       WESTERN KENTUCKY UNIVERSITY    KY  421013576   
713   00950500                  TRENT UNIVERSITY    FC  K9J   7B8   
1187  00158600             OGLETHORPE UNIVERSITY    GA  303192737   
687   00858200        WILFRID LAURIER UNIVERSITY    FC  N2L   3C5   

3                    FFEL SUBSIDIZED                        \
4        School Type      Recipients # of Loans Originated   
3742          PUBLIC            2647                  2714   
1596          PUBLIC            7432                  7510   
713   FOREIGN PUBLIC               4                     4   
1187         PRIVATE             256                   257   
687   FOREIGN PUBLIC               0                     0   

3                                                                 ...  \
4    $ of Loans Originated # of Disbursements $ of Disbursements  ...   
3742               8638250               2714            4365848  ...   
1596              34175895               7510           17278795  ...   
713                  27000                  4              15350  ...   
1187               1093010                258             560582  ...   
687                      0                  0                  0  ...   

3    FFEL PARENT PLUS                                              \
4          Recipients # of Loans Originated $ of Loans Originated   
3742              127                   128                848885   
1596              826                   867               6468300   
713                 0                     0                     0   
1187               56                    59                663811   
687                 2                     2                 35200   

3                                          FFEL GRAD PLUS  \
4    # of Disbursements $ of Disbursements     Recipients   
3742                128             445256              0   
1596                867            3278687              0   
713                   0                  0              0   
1187                 59             366462              0   
687                   2              17600              0   

3                                                                    \
4    # of Loans Originated $ of Loans Originated # of Disbursements   
3742                     0                     0                  0   
1596                     0                     0                  0   
713                      0                     0                  0   
1187                     0                     0                  0   
687                      0                     0                  0   

3                        
4    $ of Disbursements  
3742                  0  
1596                  0  
713                   0  
1187                  0  
687                   0  

[5 rows x 25 columns]

## 8. Create a copy of the data and "flatten it"
* Create a copy of this dataframe where layers are "flattened"
* The flattened dataframe will not have multi-level column names.
* This will aid plotting later on as accessing specific columns will become easier, and we can simply group together the loan type columns by matching the column name to the loan type, such as "SUBSIDIZED."
* Display flat version <u>qa_clean3</u>
* New columns should contain loan type such as "Recipients_FFEL_SUBSIDIZED".

In [13]:
#Flat version
qa_clean3 = qa_clean2.copy()
qa_clean3.columns = [f"{grp}_{col}" for grp, col in qa_clean2.columns]


In [14]:
qa_clean3

,nan_OPE ID,nan_School,nan_State,nan_Zip Code,nan_School Type,FFEL SUBSIDIZED_Recipients,FFEL SUBSIDIZED_# of Loans Originated,FFEL SUBSIDIZED_$ of Loans Originated,FFEL SUBSIDIZED_# of Disbursements,FFEL SUBSIDIZED_$ of Disbursements,...,FFEL PARENT PLUS_Recipients,FFEL PARENT PLUS_# of Loans Originated,FFEL PARENT PLUS_$ of Loans Originated,FFEL PARENT PLUS_# of Disbursements,FFEL PARENT PLUS_$ of Disbursements,FFEL GRAD PLUS_Recipients,FFEL GRAD PLUS_# of Loans Originated,FFEL GRAD PLUS_$ of Loans Originated,FFEL GRAD PLUS_# of Disbursements,FFEL GRAD PLUS_$ of Disbursements
0,00106100,ALASKA PACIFIC UNIVERSITY,AK,995084672,PRIVATE,291,291,1546994,292,830513,...,31,33,386770,35,192181,5,5,69730,5,34865
1,00106300,UNIVERSITY OF ALASKA FAIRBANKS,AK,997757500,PUBLIC,1413,1434,6394735,1455,3290699,...,36,40,265784,40,138160,4,4,68620,4,28988
2,00106500,UNIVERSITY OF ALASKA SOUTHEAST,AK,998018680,PUBLIC,406,409,1866473,439,1044946,...,8,8,88046,8,44024,0,0,0,0,0
3,01146200,UNIVERSITY OF ALASKA ANCHORAGE,AK,995088050,PUBLIC,2998,3042,12780036,3045,6440086,...,123,125,1288224,125,645683,9,11,43878,11,24063
4,02541000,ALASKA CAREER COLLEGE,AK,995071033,PROPRIETARY,38,38,103869,38,52178,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3815,00393200,UNIVERSITY OF WYOMING,WY,820713663,PUBLIC,2699,2735,12341711,2738,6819888,...,195,207,1830536,209,950682,58,65,740877,66,416236
3816,00393300,WESTERN WYOMING COMMUNITY COLLEGE,WY,829010428,PUBLIC,214,214,731883,215,359742,...,2,2,13300,2,6650,0,0,0,0,0
3817,00728900,CENTRAL WYOMING COLLEGE,WY,825012215,PUBLIC,149,154,414959,154,212783,...,7,7,34978,7,17492,0,0,0,0,0
3818,00915700,WYOTECH,WY,820729519,PROPRIETARY,1049,1099,2554580,1220,1498483,...,478,490,7567391,522,4033026,0,0,0,0,0


#### Observation: Columns 0-4 contain "nan_" at the start of the column name. This should be removed.

### Fix columns that begin with "nan"
* Remove nan prefix from columns by replacing "nan_" with nothing.
* Display the updated dataframe to check for differences

In [15]:
qa_clean3.columns = [col.replace('nan_', '') for col in qa_clean3.columns]

In [23]:
qa_clean3

,OPE ID,School,State,Zip Code,School Type,FFEL SUBSIDIZED_Recipients,FFEL SUBSIDIZED_# of Loans Originated,FFEL SUBSIDIZED_$ of Loans Originated,FFEL SUBSIDIZED_# of Disbursements,FFEL SUBSIDIZED_$ of Disbursements,...,FFEL PARENT PLUS_Recipients,FFEL PARENT PLUS_# of Loans Originated,FFEL PARENT PLUS_$ of Loans Originated,FFEL PARENT PLUS_# of Disbursements,FFEL PARENT PLUS_$ of Disbursements,FFEL GRAD PLUS_Recipients,FFEL GRAD PLUS_# of Loans Originated,FFEL GRAD PLUS_$ of Loans Originated,FFEL GRAD PLUS_# of Disbursements,FFEL GRAD PLUS_$ of Disbursements
0,00106100,ALASKA PACIFIC UNIVERSITY,AK,995084672,PRIVATE,291,291,1546994,292,830513,...,31,33,386770,35,192181,5,5,69730,5,34865
1,00106300,UNIVERSITY OF ALASKA FAIRBANKS,AK,997757500,PUBLIC,1413,1434,6394735,1455,3290699,...,36,40,265784,40,138160,4,4,68620,4,28988
2,00106500,UNIVERSITY OF ALASKA SOUTHEAST,AK,998018680,PUBLIC,406,409,1866473,439,1044946,...,8,8,88046,8,44024,0,0,0,0,0
3,01146200,UNIVERSITY OF ALASKA ANCHORAGE,AK,995088050,PUBLIC,2998,3042,12780036,3045,6440086,...,123,125,1288224,125,645683,9,11,43878,11,24063
4,02541000,ALASKA CAREER COLLEGE,AK,995071033,PROPRIETARY,38,38,103869,38,52178,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3815,00393200,UNIVERSITY OF WYOMING,WY,820713663,PUBLIC,2699,2735,12341711,2738,6819888,...,195,207,1830536,209,950682,58,65,740877,66,416236
3816,00393300,WESTERN WYOMING COMMUNITY COLLEGE,WY,829010428,PUBLIC,214,214,731883,215,359742,...,2,2,13300,2,6650,0,0,0,0,0
3817,00728900,CENTRAL WYOMING COLLEGE,WY,825012215,PUBLIC,149,154,414959,154,212783,...,7,7,34978,7,17492,0,0,0,0,0
3818,00915700,WYOTECH,WY,820729519,PROPRIETARY,1049,1099,2554580,1220,1498483,...,478,490,7567391,522,4033026,0,0,0,0,0


#### Success: Columns 0-4 contain normal names instead of "naan_" prefix

## 9. Delete rows containing foreign schools as we will only be examining US schools
### Ways to detect:
* "State" column contains value "FC" (Foreign College)
* "School Type" contains value "FOREIGN"
### We will use a command to search for either of the above conditions in order to determine if a school is foreign.
1.  Index the column by matching the column information to either condition above
* Use ~() to negate or perform the opposite of the conditions (include only non-foreign schools)
* Use .str.upper() to force the column name to uppercase in order to match with the "FOREIGN" string
* Keep only US schools
  
2.  Save to new dataframe, <u>qa_clean4</u>
3.  Display new dataframe to check for foreign schools
4.  Run a command to check for foreign schools

In [16]:
qa_clean4 = qa_clean3[
    ~((qa_clean3['State'] == 'FC') | (qa_clean3['School Type'].str.upper() == 'FOREIGN'))
]

In [26]:
qa_clean4

,OPE ID,School,State,Zip Code,School Type,FFEL SUBSIDIZED_Recipients,FFEL SUBSIDIZED_# of Loans Originated,FFEL SUBSIDIZED_$ of Loans Originated,FFEL SUBSIDIZED_# of Disbursements,FFEL SUBSIDIZED_$ of Disbursements,...,FFEL PARENT PLUS_Recipients,FFEL PARENT PLUS_# of Loans Originated,FFEL PARENT PLUS_$ of Loans Originated,FFEL PARENT PLUS_# of Disbursements,FFEL PARENT PLUS_$ of Disbursements,FFEL GRAD PLUS_Recipients,FFEL GRAD PLUS_# of Loans Originated,FFEL GRAD PLUS_$ of Loans Originated,FFEL GRAD PLUS_# of Disbursements,FFEL GRAD PLUS_$ of Disbursements
0,00106100,ALASKA PACIFIC UNIVERSITY,AK,995084672,PRIVATE,291,291,1546994,292,830513,...,31,33,386770,35,192181,5,5,69730,5,34865
1,00106300,UNIVERSITY OF ALASKA FAIRBANKS,AK,997757500,PUBLIC,1413,1434,6394735,1455,3290699,...,36,40,265784,40,138160,4,4,68620,4,28988
2,00106500,UNIVERSITY OF ALASKA SOUTHEAST,AK,998018680,PUBLIC,406,409,1866473,439,1044946,...,8,8,88046,8,44024,0,0,0,0,0
3,01146200,UNIVERSITY OF ALASKA ANCHORAGE,AK,995088050,PUBLIC,2998,3042,12780036,3045,6440086,...,123,125,1288224,125,645683,9,11,43878,11,24063
4,02541000,ALASKA CAREER COLLEGE,AK,995071033,PROPRIETARY,38,38,103869,38,52178,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3815,00393200,UNIVERSITY OF WYOMING,WY,820713663,PUBLIC,2699,2735,12341711,2738,6819888,...,195,207,1830536,209,950682,58,65,740877,66,416236
3816,00393300,WESTERN WYOMING COMMUNITY COLLEGE,WY,829010428,PUBLIC,214,214,731883,215,359742,...,2,2,13300,2,6650,0,0,0,0,0
3817,00728900,CENTRAL WYOMING COLLEGE,WY,825012215,PUBLIC,149,154,414959,154,212783,...,7,7,34978,7,17492,0,0,0,0,0
3818,00915700,WYOTECH,WY,820729519,PROPRIETARY,1049,1099,2554580,1220,1498483,...,478,490,7567391,522,4033026,0,0,0,0,0


#### Observation: Foreign schools appear to be removed, but we forgot to check for Puerto Rican schools (PR).
* Use .unique() to see unique states, to check if FC (foreign college) or PR (Puerto Rico) appear. 
* Use .unique() to see unique school types, to check if foreign schools still exist.
* If either still exist, then foreign schools are still in the data and must be removed.

In [27]:
print("Unique states:", qa_clean4['State'].unique())
print("Unique school types:", qa_clean4['School Type'].unique())


Unique states: ['AK' 'AL' 'AR' 'AZ' 'CA' 'CO' 'CT' 'DC' 'DE' 'FL' 'GA' 'GU' 'HI' 'IA'
 'ID' 'IL' 'IN' 'KS' 'KY' 'LA' 'MA' 'MD' 'ME' 'MI' 'MN' 'MO' 'MS' 'MT'
 'NC' 'ND' 'NE' 'NH' 'NJ' 'NM' 'NR' 'NV' 'NY' 'OH' 'OK' 'OR' 'PA' 'PR'
 'RI' 'SC' 'SD' 'TN' 'TX' 'UT' 'VA' 'VT' 'WA' 'WI' 'WV' 'WY']
Unique school types: ['PRIVATE' 'PUBLIC' 'PROPRIETARY' 'OTHER']


#### Observation: PR schools still exist, so they will be dropped.
* Use the same command (negating the index of the foreign state) to keep only schools that are not in PR (US schools)
* Check for unique states again
* Check new data overall

In [17]:
qa_clean4 = qa_clean3[
    ~((qa_clean3['State'] == 'PR'))
]

In [18]:
print("Unique states:", qa_clean4['State'].unique())

Unique states: ['AK' 'AL' 'AR' 'AZ' 'CA' 'CO' 'CT' 'DC' 'DE' 'FC' 'FL' 'GA' 'GU' 'HI'
 'IA' 'ID' 'IL' 'IN' 'KS' 'KY' 'LA' 'MA' 'MD' 'ME' 'MI' 'MN' 'MO' 'MS'
 'MT' 'NC' 'ND' 'NE' 'NH' 'NJ' 'NM' 'NR' 'NV' 'NY' 'OH' 'OK' 'OR' 'PA'
 'RI' 'SC' 'SD' 'TN' 'TX' 'UT' 'VA' 'VT' 'WA' 'WI' 'WV' 'WY']


In [30]:
qa_clean4

,OPE ID,School,State,Zip Code,School Type,FFEL SUBSIDIZED_Recipients,FFEL SUBSIDIZED_# of Loans Originated,FFEL SUBSIDIZED_$ of Loans Originated,FFEL SUBSIDIZED_# of Disbursements,FFEL SUBSIDIZED_$ of Disbursements,...,FFEL PARENT PLUS_Recipients,FFEL PARENT PLUS_# of Loans Originated,FFEL PARENT PLUS_$ of Loans Originated,FFEL PARENT PLUS_# of Disbursements,FFEL PARENT PLUS_$ of Disbursements,FFEL GRAD PLUS_Recipients,FFEL GRAD PLUS_# of Loans Originated,FFEL GRAD PLUS_$ of Loans Originated,FFEL GRAD PLUS_# of Disbursements,FFEL GRAD PLUS_$ of Disbursements
0,00106100,ALASKA PACIFIC UNIVERSITY,AK,995084672,PRIVATE,291,291,1546994,292,830513,...,31,33,386770,35,192181,5,5,69730,5,34865
1,00106300,UNIVERSITY OF ALASKA FAIRBANKS,AK,997757500,PUBLIC,1413,1434,6394735,1455,3290699,...,36,40,265784,40,138160,4,4,68620,4,28988
2,00106500,UNIVERSITY OF ALASKA SOUTHEAST,AK,998018680,PUBLIC,406,409,1866473,439,1044946,...,8,8,88046,8,44024,0,0,0,0,0
3,01146200,UNIVERSITY OF ALASKA ANCHORAGE,AK,995088050,PUBLIC,2998,3042,12780036,3045,6440086,...,123,125,1288224,125,645683,9,11,43878,11,24063
4,02541000,ALASKA CAREER COLLEGE,AK,995071033,PROPRIETARY,38,38,103869,38,52178,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3815,00393200,UNIVERSITY OF WYOMING,WY,820713663,PUBLIC,2699,2735,12341711,2738,6819888,...,195,207,1830536,209,950682,58,65,740877,66,416236
3816,00393300,WESTERN WYOMING COMMUNITY COLLEGE,WY,829010428,PUBLIC,214,214,731883,215,359742,...,2,2,13300,2,6650,0,0,0,0,0
3817,00728900,CENTRAL WYOMING COLLEGE,WY,825012215,PUBLIC,149,154,414959,154,212783,...,7,7,34978,7,17492,0,0,0,0,0
3818,00915700,WYOTECH,WY,820729519,PROPRIETARY,1049,1099,2554580,1220,1498483,...,478,490,7567391,522,4033026,0,0,0,0,0


#### Observation: PR schools do not exist in data anymore

In [31]:
qa_clean4.columns

Index(['OPE ID', 'School', 'State', 'Zip Code', 'School Type',
       'FFEL SUBSIDIZED_Recipients', 'FFEL SUBSIDIZED_# of Loans Originated',
       'FFEL SUBSIDIZED_$ of Loans Originated',
       'FFEL SUBSIDIZED_# of Disbursements',
       'FFEL SUBSIDIZED_$ of Disbursements', 'FFEL UNSUBSIDIZED_Recipients',
       'FFEL UNSUBSIDIZED_# of Loans Originated',
       'FFEL UNSUBSIDIZED_$ of Loans Originated',
       'FFEL UNSUBSIDIZED_# of Disbursements',
       'FFEL UNSUBSIDIZED_$ of Disbursements', 'FFEL PARENT PLUS_Recipients',
       'FFEL PARENT PLUS_# of Loans Originated',
       'FFEL PARENT PLUS_$ of Loans Originated',
       'FFEL PARENT PLUS_# of Disbursements',
       'FFEL PARENT PLUS_$ of Disbursements', 'FFEL GRAD PLUS_Recipients',
       'FFEL GRAD PLUS_# of Loans Originated',
       'FFEL GRAD PLUS_$ of Loans Originated',
       'FFEL GRAD PLUS_# of Disbursements',
       'FFEL GRAD PLUS_$ of Disbursements'],
      dtype='object')

In [19]:
#check for community colleges
qa_clean4[qa_clean4['School'].str.contains('COMMUNITY', case=False, na=False)]['School'].unique()

array(['CENTRAL ALABAMA COMMUNITY COLLEGE',
       'ENTERPRISE STATE COMMUNITY COLLEGE',
       'NORTHEAST ALABAMA COMMUNITY COLLEGE',
       'SOUTHERN UNION STATE COMMUNITY COLLEGE',
       'JAMES H. FAULKNER STATE COMMUNITY COLLEGE',
       'NORTHWEST - SHOALS COMMUNITY COLLEGE',
       'GEORGE C. WALLACE STATE COMMUNITY COLLEGE',
       'UNIVERSITY OF ARKANSAS COMMUNITY COLLEGE AT MORRILTON',
       'NATIONAL PARK COMMUNITY COLLEGE',
       'UNIVERSITY OF ARKANSAS COMMUNITY COLLEGE AT BATESVILLE',
       'SOUTH ARKANSAS COMMUNITY COLLEGE',
       'NORTHWEST ARKANSAS COMMUNITY COLLEGE',
       'GLENDALE COMMUNITY COLLEGE', 'MESA COMMUNITY COLLEGE',
       'PIMA COUNTY COMMUNITY COLLEGE', 'GATEWAY COMMUNITY COLLEGE',
       'SCOTTSDALE COMMUNITY COLLEGE', 'SOUTH MOUNTAIN COMMUNITY COLLEGE',
       'RIO SALADO COMMUNITY COLLEGE',
       'PARADISE VALLEY COMMUNITY COLLEGE',
       'COCONINO COUNTY COMMUNITY COLLEGE',
       'ESTRELLA MOUNTAIN COMMUNITY COLLEGE',
       'CERRITOS COMMUNI

## 10. Delete rows containing all missing values, which are marked with "0"
* Missing values have been replaced with "0", so it is difficult to discern true 0 values from missing values
* For this reason, we will only be removing rows that contain multiple zeroes as they are true outliers to the data
### CLeaning process:
1. Identify Numeric Columns (from column 5 onward), so that we only alter numbers
2. Count Zeroes in Each Row (only in numeric columns)
3. Drop rows containing 2 or more zeroes, as this is a strong indicator of missing values (due to missing values being converted to 0)
* Reset index to discard old index structure instead of adding it as a new column
* Save this to a new dataframe: <u>qa_clean5</u>

4. Check the changes by counting the amount of rows in the new data set. Count the amount of dropped rows by finding the amount of rows containing 2 or more zeroes.
* Use len(dataframe) to determine number of remaining rows
* Use len(zero_counts[zero+counts>1]) to determine number of dropped rows

5. Display new data, <u>qa_clean5</u>

In [32]:
numeric_cols = qa_clean4.columns[5:]

In [33]:
zero_counts = (qa_clean4[numeric_cols] == 0).sum(axis=1)


In [34]:
qa_clean5 = qa_clean4[zero_counts <= 1].reset_index(drop=True)


In [35]:
print(f"Remaining rows: {len(qa_clean5)}")
print(f"Dropped rows with >1 zero: {len(zero_counts[zero_counts > 1])}")


Remaining rows: 652
Dropped rows with >1 zero: 3143


## Display Cleaned Data

In [36]:
qa_clean5

,OPE ID,School,State,Zip Code,School Type,FFEL SUBSIDIZED_Recipients,FFEL SUBSIDIZED_# of Loans Originated,FFEL SUBSIDIZED_$ of Loans Originated,FFEL SUBSIDIZED_# of Disbursements,FFEL SUBSIDIZED_$ of Disbursements,...,FFEL PARENT PLUS_Recipients,FFEL PARENT PLUS_# of Loans Originated,FFEL PARENT PLUS_$ of Loans Originated,FFEL PARENT PLUS_# of Disbursements,FFEL PARENT PLUS_$ of Disbursements,FFEL GRAD PLUS_Recipients,FFEL GRAD PLUS_# of Loans Originated,FFEL GRAD PLUS_$ of Loans Originated,FFEL GRAD PLUS_# of Disbursements,FFEL GRAD PLUS_$ of Disbursements
0,00106100,ALASKA PACIFIC UNIVERSITY,AK,995084672,PRIVATE,291,291,1546994,292,830513,...,31,33,386770,35,192181,5,5,69730,5,34865
1,00106300,UNIVERSITY OF ALASKA FAIRBANKS,AK,997757500,PUBLIC,1413,1434,6394735,1455,3290699,...,36,40,265784,40,138160,4,4,68620,4,28988
2,01146200,UNIVERSITY OF ALASKA ANCHORAGE,AK,995088050,PUBLIC,2998,3042,12780036,3045,6440086,...,123,125,1288224,125,645683,9,11,43878,11,24063
3,00100200,ALABAMA AGRICULTURAL & MECHANICAL UNIVERSITY,AL,357621357,PUBLIC,2675,2692,12343721,2701,6178188,...,231,236,1744374,239,927673,2,2,9524,2,6262
4,00100300,FAULKNER UNIVERSITY,AL,361093398,PRIVATE,1392,1403,6244282,1426,3470561,...,78,78,611850,79,347361,188,203,4204847,208,2143612
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
647,00380600,ALDERSON BROADDUS COLLEGE,WV,264161051,PRIVATE,322,327,1430393,327,724089,...,43,46,362372,46,186638,17,19,189386,19,107549
648,00380700,MOUNTAIN STATE UNIVERSITY,WV,258015624,PRIVATE,1285,1288,3299331,1400,1812988,...,6,6,30348,6,15175,59,59,440493,61,228951
649,00381800,UNIVERSITY OF CHARLESTON,WV,253041099,PRIVATE,534,544,3220394,551,1658291,...,19,21,174804,21,87402,100,107,1502249,107,757897
650,00383100,WHEELING JESUIT UNIVERSITY,WV,260036243,PRIVATE,642,646,2841866,648,1519125,...,79,82,816500,83,421514,25,25,265555,25,163898


### Observation: Many rows have been removed. 
* Foreign schools were dropped earlier, including Puerto Rican schools
* Rows with a high probability of containing missing values (more than 2 0's) have been dropped.
* This dataframe, <u>qa_clean5</u>, should now be clean enough to perform descriptive statistics.

In [37]:
qa_clean5.describe()

,FFEL SUBSIDIZED_Recipients,FFEL SUBSIDIZED_# of Loans Originated,FFEL SUBSIDIZED_$ of Loans Originated,FFEL SUBSIDIZED_# of Disbursements,FFEL SUBSIDIZED_$ of Disbursements,FFEL UNSUBSIDIZED_Recipients,FFEL UNSUBSIDIZED_# of Loans Originated,FFEL UNSUBSIDIZED_$ of Loans Originated,FFEL UNSUBSIDIZED_# of Disbursements,FFEL UNSUBSIDIZED_$ of Disbursements,FFEL PARENT PLUS_Recipients,FFEL PARENT PLUS_# of Loans Originated,FFEL PARENT PLUS_$ of Loans Originated,FFEL PARENT PLUS_# of Disbursements,FFEL PARENT PLUS_$ of Disbursements,FFEL GRAD PLUS_Recipients,FFEL GRAD PLUS_# of Loans Originated,FFEL GRAD PLUS_$ of Loans Originated,FFEL GRAD PLUS_# of Disbursements,FFEL GRAD PLUS_$ of Disbursements
count,652.000000,652.000000,6.520000e+02,652.000000,6.520000e+02,652.000000,652.000000,6.520000e+02,652.000000,6.520000e+02,652.000000,652.000000,6.520000e+02,652.000000,6.520000e+02,652.000000,652.000000,6.520000e+02,652.000000,6.520000e+02
mean,2330.081288,2370.616564,1.177131e+07,2402.329755,5.969116e+06,2339.343558,2487.179448,1.489701e+07,2521.185583,7.509986e+06,237.487730,246.404908,3.309502e+06,249.452454,1.686973e+06,140.760736,152.078221,2.938906e+06,154.605828,1.447169e+06
std,3795.671723,3859.537532,1.778915e+07,3960.662386,9.349229e+06,3666.881070,3847.366240,2.365699e+07,3954.170775,1.236648e+07,353.176659,366.716106,5.849507e+06,369.933729,2.986120e+06,360.067861,391.411803,9.529597e+06,397.746756,4.718619e+06
min,1.000000,1.000000,4.315000e+03,1.000000,3.440000e+03,1.000000,1.000000,1.200000e+04,1.000000,6.000000e+03,1.000000,1.000000,6.310000e+03,1.000000,3.155000e+03,1.000000,1.000000,3.410000e+02,1.000000,2.900000e+02
25%,488.250000,491.500000,2.465548e+06,499.000000,1.288062e+06,463.250000,485.750000,2.635020e+06,490.250000,1.341806e+06,27.000000,27.000000,2.857065e+05,27.750000,1.393585e+05,3.000000,3.000000,2.885450e+04,3.750000,1.418275e+04
50%,1244.500000,1255.000000,6.187954e+06,1277.000000,3.157526e+06,1269.000000,1382.000000,7.289778e+06,1386.000000,3.743064e+06,121.000000,124.000000,1.437626e+06,125.000000,7.350180e+05,16.000000,17.000000,1.852235e+05,17.500000,9.372900e+04
75%,2859.250000,2932.000000,1.464657e+07,2960.250000,7.284365e+06,2850.000000,2997.000000,1.878274e+07,3029.000000,9.298971e+06,301.250000,313.750000,4.054463e+06,316.000000,2.040820e+06,86.500000,93.000000,1.219552e+06,94.000000,6.142980e+05
max,67590.000000,68099.000000,2.833224e+08,70312.000000,1.548300e+08,62730.000000,63556.000000,3.750060e+08,65830.000000,1.985650e+08,3272.000000,3339.000000,8.802681e+07,3354.000000,4.467873e+07,4112.000000,4286.000000,1.311844e+08,4323.000000,6.673883e+07


### Observation: None of the values in the table appear to be 0. This means empty rows have likely been dropped successfully.
* Our descriptive statistics are now more accurate.

In [38]:
qa_clean5

,OPE ID,School,State,Zip Code,School Type,FFEL SUBSIDIZED_Recipients,FFEL SUBSIDIZED_# of Loans Originated,FFEL SUBSIDIZED_$ of Loans Originated,FFEL SUBSIDIZED_# of Disbursements,FFEL SUBSIDIZED_$ of Disbursements,...,FFEL PARENT PLUS_Recipients,FFEL PARENT PLUS_# of Loans Originated,FFEL PARENT PLUS_$ of Loans Originated,FFEL PARENT PLUS_# of Disbursements,FFEL PARENT PLUS_$ of Disbursements,FFEL GRAD PLUS_Recipients,FFEL GRAD PLUS_# of Loans Originated,FFEL GRAD PLUS_$ of Loans Originated,FFEL GRAD PLUS_# of Disbursements,FFEL GRAD PLUS_$ of Disbursements
0,00106100,ALASKA PACIFIC UNIVERSITY,AK,995084672,PRIVATE,291,291,1546994,292,830513,...,31,33,386770,35,192181,5,5,69730,5,34865
1,00106300,UNIVERSITY OF ALASKA FAIRBANKS,AK,997757500,PUBLIC,1413,1434,6394735,1455,3290699,...,36,40,265784,40,138160,4,4,68620,4,28988
2,01146200,UNIVERSITY OF ALASKA ANCHORAGE,AK,995088050,PUBLIC,2998,3042,12780036,3045,6440086,...,123,125,1288224,125,645683,9,11,43878,11,24063
3,00100200,ALABAMA AGRICULTURAL & MECHANICAL UNIVERSITY,AL,357621357,PUBLIC,2675,2692,12343721,2701,6178188,...,231,236,1744374,239,927673,2,2,9524,2,6262
4,00100300,FAULKNER UNIVERSITY,AL,361093398,PRIVATE,1392,1403,6244282,1426,3470561,...,78,78,611850,79,347361,188,203,4204847,208,2143612
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
647,00380600,ALDERSON BROADDUS COLLEGE,WV,264161051,PRIVATE,322,327,1430393,327,724089,...,43,46,362372,46,186638,17,19,189386,19,107549
648,00380700,MOUNTAIN STATE UNIVERSITY,WV,258015624,PRIVATE,1285,1288,3299331,1400,1812988,...,6,6,30348,6,15175,59,59,440493,61,228951
649,00381800,UNIVERSITY OF CHARLESTON,WV,253041099,PRIVATE,534,544,3220394,551,1658291,...,19,21,174804,21,87402,100,107,1502249,107,757897
650,00383100,WHEELING JESUIT UNIVERSITY,WV,260036243,PRIVATE,642,646,2841866,648,1519125,...,79,82,816500,83,421514,25,25,265555,25,163898


#### Observation: The data contains significantly less rows than the original, meaning that we have dropped many rows.

## Data Cleaning: Save new data frame
* We will copy this dataframe under a new name so that we can easily tell that it has been cleaned properly and is the final version
* The new dataframe will be named "qa_cleaned"
* Display the new dataframe
* At the end of the file, we will save all of these cleaned dataframes to CSV files so that they can be opened in the next project.

In [39]:
qa_cleaned = qa_clean5.copy()

In [40]:
qa_cleaned

,OPE ID,School,State,Zip Code,School Type,FFEL SUBSIDIZED_Recipients,FFEL SUBSIDIZED_# of Loans Originated,FFEL SUBSIDIZED_$ of Loans Originated,FFEL SUBSIDIZED_# of Disbursements,FFEL SUBSIDIZED_$ of Disbursements,...,FFEL PARENT PLUS_Recipients,FFEL PARENT PLUS_# of Loans Originated,FFEL PARENT PLUS_$ of Loans Originated,FFEL PARENT PLUS_# of Disbursements,FFEL PARENT PLUS_$ of Disbursements,FFEL GRAD PLUS_Recipients,FFEL GRAD PLUS_# of Loans Originated,FFEL GRAD PLUS_$ of Loans Originated,FFEL GRAD PLUS_# of Disbursements,FFEL GRAD PLUS_$ of Disbursements
0,00106100,ALASKA PACIFIC UNIVERSITY,AK,995084672,PRIVATE,291,291,1546994,292,830513,...,31,33,386770,35,192181,5,5,69730,5,34865
1,00106300,UNIVERSITY OF ALASKA FAIRBANKS,AK,997757500,PUBLIC,1413,1434,6394735,1455,3290699,...,36,40,265784,40,138160,4,4,68620,4,28988
2,01146200,UNIVERSITY OF ALASKA ANCHORAGE,AK,995088050,PUBLIC,2998,3042,12780036,3045,6440086,...,123,125,1288224,125,645683,9,11,43878,11,24063
3,00100200,ALABAMA AGRICULTURAL & MECHANICAL UNIVERSITY,AL,357621357,PUBLIC,2675,2692,12343721,2701,6178188,...,231,236,1744374,239,927673,2,2,9524,2,6262
4,00100300,FAULKNER UNIVERSITY,AL,361093398,PRIVATE,1392,1403,6244282,1426,3470561,...,78,78,611850,79,347361,188,203,4204847,208,2143612
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
647,00380600,ALDERSON BROADDUS COLLEGE,WV,264161051,PRIVATE,322,327,1430393,327,724089,...,43,46,362372,46,186638,17,19,189386,19,107549
648,00380700,MOUNTAIN STATE UNIVERSITY,WV,258015624,PRIVATE,1285,1288,3299331,1400,1812988,...,6,6,30348,6,15175,59,59,440493,61,228951
649,00381800,UNIVERSITY OF CHARLESTON,WV,253041099,PRIVATE,534,544,3220394,551,1658291,...,19,21,174804,21,87402,100,107,1502249,107,757897
650,00383100,WHEELING JESUIT UNIVERSITY,WV,260036243,PRIVATE,642,646,2841866,648,1519125,...,79,82,816500,83,421514,25,25,265555,25,163898


# <u>Data Cleaning: Award Year Summary (AYS)</u>
1. Examine both dataframes for improper column names or missing rows/values
2. Use .describe() to find descriptive statistics on each dataframe. 

## 1. Examine dataframe for immediate visible isues
* ays_clean: our copy of the dataframe so that we may easily retrieve the original dataframe incase of error

In [41]:
ays_clean

,2009-2010 Award Year FFEL Volume by School,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20,Unnamed: 21,Unnamed: 22,Unnamed: 23,Unnamed: 24
0,Award Year Cumulative Activity through Quarter...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Data Run: 4/5/2012,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,FFEL SUBSIDIZED,NaN,NaN,NaN,NaN,...,FFEL PARENT PLUS,NaN,NaN,NaN,NaN,FFEL GRAD PLUS,NaN,NaN,NaN,NaN
4,OPE ID,School,State,Zip Code,School Type,Recipients,# of Loans Originated,$ of Loans Originated,# of Disbursements,$ of Disbursements,...,Recipients,# of Loans Originated,$ of Loans Originated,# of Disbursements,$ of Disbursements,Recipients,# of Loans Originated,$ of Loans Originated,# of Disbursements,$ of Disbursements
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3820,00393200,UNIVERSITY OF WYOMING,WY,820713663,PUBLIC,2699,2735,12341711,2738,6819888,...,195,207,1830536,209,950682,58,65,740877,66,416236
3821,00393300,WESTERN WYOMING COMMUNITY COLLEGE,WY,829010428,PUBLIC,214,214,731883,215,359742,...,2,2,13300,2,6650,0,0,0,0,0
3822,00728900,CENTRAL WYOMING COLLEGE,WY,825012215,PUBLIC,149,154,414959,154,212783,...,7,7,34978,7,17492,0,0,0,0,0
3823,00915700,WYOTECH,WY,820729519,PROPRIETARY,1049,1099,2554580,1220,1498483,...,478,490,7567391,522,4033026,0,0,0,0,0


### Observations:
1. Contains the same formatting issues as the QA dataframe:
* improper column names,
* leading rows are entirely missing
* missing values are entered as "0"
* data type of columns are likely incorrect (all objects instead of objects and integers)

2. Contains foreign schools


## Data Cleaning Plan:
1. Clean headers and column names
2. flatten the data and make a copy (ays_clean2)

## 1. Clean headers and column names
### 1. Extract and store groupings (row index 3) for loan types "FFEL SUBSIDIZED, FFEL UNSUBSIDIZED" etc.
* Use iloc[] to select the grouping row
* Display the grouping row, which should show types of loans

In [42]:
group_row = ays_clean.iloc[3]  #This is the grouping row (Exc: "FFEL SUBSIDIZED")

group_row #Check for FFEL SUBSIDIZED and NaN

2009-2010 Award Year FFEL Volume by School                  NaN
Unnamed: 1                                                  NaN
Unnamed: 2                                                  NaN
Unnamed: 3                                                  NaN
Unnamed: 4                                                  NaN
Unnamed: 5                                      FFEL SUBSIDIZED
Unnamed: 6                                                  NaN
Unnamed: 7                                                  NaN
Unnamed: 8                                                  NaN
Unnamed: 9                                                  NaN
Unnamed: 10                                   FFEL UNSUBSIDIZED
Unnamed: 11                                                 NaN
Unnamed: 12                                                 NaN
Unnamed: 13                                                 NaN
Unnamed: 14                                                 NaN
Unnamed: 15                             

### 2. Extract actual column names (row index 4)
* Use .iloc[] to select the row that contains proper column names
* Display the proper column names into a row called column_row

In [43]:
column_row = ays_clean.iloc[4]  #This contains: OPE ID, School Name, etc.
column_row #We should see column names and datatype object

2009-2010 Award Year FFEL Volume by School                   OPE ID
Unnamed: 1                                                   School
Unnamed: 2                                                    State
Unnamed: 3                                                 Zip Code
Unnamed: 4                                              School Type
Unnamed: 5                                               Recipients
Unnamed: 6                                    # of Loans Originated
Unnamed: 7                                    $ of Loans Originated
Unnamed: 8                                       # of Disbursements
Unnamed: 9                                       $ of Disbursements
Unnamed: 10                                              Recipients
Unnamed: 11                                   # of Loans Originated
Unnamed: 12                                   $ of Loans Originated
Unnamed: 13                                      # of Disbursements
Unnamed: 14                                     

### 3. Combine the grouping and actual column names into a multi-level index
* Replace NaNs in the group row with forward fill (to fill in loan group names)
* Display the filled names. We can now see which columns cover which loan type.
* Combine group + columnn name row into a MultiIndex

In [44]:
#Replace NaNs in the group row with forward fill (to fill in group names)
group_row_filled = group_row.fillna(method='ffill')
group_row_filled
#Combine group and column row into a MultiIndex
ays_clean.columns = pd.MultiIndex.from_arrays([group_row_filled, column_row])

C:\Users\pmost\AppData\Local\Temp\ipykernel_7892\1262388059.py:2: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  group_row_filled = group_row.fillna(method='ffill')


### 4. Examine the first 5 rows

In [45]:
ays_clean.head(5)

3                                                NaN                           \
4                                             OPE ID  School  State  Zip Code   
0  Award Year Cumulative Activity through Quarter...     NaN    NaN       NaN   
1                                 Data Run: 4/5/2012     NaN    NaN       NaN   
2                                                        NaN    NaN       NaN   
3                                                NaN     NaN    NaN       NaN   
4                                             OPE ID  School  State  Zip Code   

3               FFEL SUBSIDIZED                                                \
4  School Type       Recipients  # of Loans Originated  $ of Loans Originated   
0          NaN              NaN                    NaN                    NaN   
1          NaN              NaN                    NaN                    NaN   
2          NaN              NaN                    NaN                    NaN   
3          NaN  FFEL SUBSIDIZED                    NaN                    NaN   
4  School Type       Recipients  # of Loans Originated  $ of Loans Originated   

3                                          ...  FFEL PARENT PLUS  \
4  # of Disbursements  $ of Disbursements  ...        Recipients   
0                 NaN                 NaN  ...               NaN   
1                 NaN                 NaN  ...               NaN   
2                 NaN                 NaN  ...               NaN   
3                 NaN                 NaN  ...  FFEL PARENT PLUS   
4  # of Disbursements  $ of Disbursements  ...        Recipients   

3                                                                    \
4  # of Loans Originated  $ of Loans Originated  # of Disbursements   
0                    NaN                    NaN                 NaN   
1                    NaN                    NaN                 NaN   
2                    NaN                    NaN                 NaN   
3                    NaN                    NaN                 NaN   
4  # of Loans Originated  $ of Loans Originated  # of Disbursements   

3                      FFEL GRAD PLUS                         \
4  $ of Disbursements      Recipients  # of Loans Originated   
0                 NaN             NaN                    NaN   
1                 NaN             NaN                    NaN   
2                 NaN             NaN                    NaN   
3                 NaN  FFEL GRAD PLUS                    NaN   
4  $ of Disbursements      Recipients  # of Loans Originated   

3                                                                 
4  $ of Loans Originated  # of Disbursements  $ of Disbursements  
0                    NaN                 NaN                 NaN  
1                    NaN                 NaN                 NaN  
2                    NaN                 NaN                 NaN  
3                    NaN                 NaN                 NaN  
4  $ of Loans Originated  # of Disbursements  $ of Disbursements  

[5 rows x 25 columns]

### 5. Drop the first 5 rows (missing values + headers)
* Use .iloc[] and slicing (:) to extract the first 5 rows.
* Starting at the 5th row (where the actual column names are), we sill select the remaining rows and reset the index to reformat the dataframe.
* Save to new dataframe <u>ays_clean2</u>
* Inspect the dataframe for proper columns

In [46]:
ays_clean2 = ays_clean.iloc[5:].reset_index(drop=True)
ays_clean2.head(5)

3       NaN                                                                \
4    OPE ID                          School State   Zip Code  School Type   
0  00106100       ALASKA PACIFIC UNIVERSITY    AK  995084672      PRIVATE   
1  00106300  UNIVERSITY OF ALASKA FAIRBANKS    AK  997757500       PUBLIC   
2  00106500  UNIVERSITY OF ALASKA SOUTHEAST    AK  998018680       PUBLIC   
3  01146200  UNIVERSITY OF ALASKA ANCHORAGE    AK  995088050       PUBLIC   
4  02541000           ALASKA CAREER COLLEGE    AK  995071033  PROPRIETARY   

3 FFEL SUBSIDIZED                                              \
4      Recipients # of Loans Originated $ of Loans Originated   
0             291                   291               1546994   
1            1413                  1434               6394735   
2             406                   409               1866473   
3            2998                  3042              12780036   
4              38                    38                103869   

3                                        ... FFEL PARENT PLUS  \
4 # of Disbursements $ of Disbursements  ...       Recipients   
0                292             830513  ...               31   
1               1455            3290699  ...               36   
2                439            1044946  ...                8   
3               3045            6440086  ...              123   
4                 38              52178  ...                0   

3                                                                 \
4 # of Loans Originated $ of Loans Originated # of Disbursements   
0                    33                386770                 35   
1                    40                265784                 40   
2                     8                 88046                  8   
3                   125               1288224                125   
4                     0                     0                  0   

3                    FFEL GRAD PLUS                        \
4 $ of Disbursements     Recipients # of Loans Originated   
0             192181              5                     5   
1             138160              4                     4   
2              44024              0                     0   
3             645683              9                    11   
4                  0              0                     0   

3                                                              
4 $ of Loans Originated # of Disbursements $ of Disbursements  
0                 69730                  5              34865  
1                 68620                  4              28988  
2                     0                  0                  0  
3                 43878                 11              24063  
4                     0                  0                  0  

[5 rows x 25 columns]

### Now the dataframe (qa_clean2) has multi-level column names:
* Level 0: Loan category (FFEL SUBSIDIZED)
* Level 1: Actual column names ("School", etc.)

#### Multi-index columns
* The columns appear as: ('FFEL SUBSIDIZED', 'Loan Volume')
* The columns can be accessed like so: qa_clean2[('FFEL SUBSIDIZED', 'Loan Volume')]

### Improvements:
* Proper column names
* Subcategory of loan types within column index
* Rows filled with missing values have been deleted
### Further cleaning is needed:
* Higher-level columns in the index still contain a missing value (NaN) for columns 0-4
* Convert values (only the numbers) to numeric type so we can perform operations on them
* Delete rows containing foreign schools (school type contains the word "FOREIGN") as we will only be examining US schools
* Check for missing values ("0"s).
* Delete rows containing missing values, which are marked with "0"
* Create a copy of this dataframe where layers are "flattened", so the dataframe does not have multi-level column names. This will aid plotting later on as accessing specific columns will become easier, and we can simply group together the loan type columns by matching the column name to the loan type, such as "SUBSIDIZED."


### 6. Convert numbers to numeric
* Only change the columns from index 5 onward, as columns 0-4 contain letters and should not be changed to numbers.
* Display the data
* Use .dtypes() to check that the columns have been converted to numeric types
* use .describe() on the general data to extract descriptive statistics

In [47]:
for col in ays_clean2.columns[5:]:
    ays_clean2[col] = pd.to_numeric(ays_clean2[col], errors='coerce')

In [48]:
ays_clean2

3          NaN                                                      \
4       OPE ID                             School State   Zip Code   
0     00106100          ALASKA PACIFIC UNIVERSITY    AK  995084672   
1     00106300     UNIVERSITY OF ALASKA FAIRBANKS    AK  997757500   
2     00106500     UNIVERSITY OF ALASKA SOUTHEAST    AK  998018680   
3     01146200     UNIVERSITY OF ALASKA ANCHORAGE    AK  995088050   
4     02541000              ALASKA CAREER COLLEGE    AK  995071033   
...        ...                                ...   ...        ...   
3815  00393200              UNIVERSITY OF WYOMING    WY  820713663   
3816  00393300  WESTERN WYOMING COMMUNITY COLLEGE    WY  829010428   
3817  00728900            CENTRAL WYOMING COLLEGE    WY  825012215   
3818  00915700                            WYOTECH    WY  820729519   
3819  00925900   LARAMIE COUNTY COMMUNITY COLLEGE    WY  820073299   

3                 FFEL SUBSIDIZED                                              \
4     School Type      Recipients # of Loans Originated $ of Loans Originated   
0         PRIVATE             291                   291               1546994   
1          PUBLIC            1413                  1434               6394735   
2          PUBLIC             406                   409               1866473   
3          PUBLIC            2998                  3042              12780036   
4     PROPRIETARY              38                    38                103869   
...           ...             ...                   ...                   ...   
3815       PUBLIC            2699                  2735              12341711   
3816       PUBLIC             214                   214                731883   
3817       PUBLIC             149                   154                414959   
3818  PROPRIETARY            1049                  1099               2554580   
3819       PUBLIC             281                   291                885794   

3                                           ... FFEL PARENT PLUS  \
4    # of Disbursements $ of Disbursements  ...       Recipients   
0                   292             830513  ...               31   
1                  1455            3290699  ...               36   
2                   439            1044946  ...                8   
3                  3045            6440086  ...              123   
4                    38              52178  ...                0   
...                 ...                ...  ...              ...   
3815               2738            6819888  ...              195   
3816                215             359742  ...                2   
3817                154             212783  ...                7   
3818               1220            1498483  ...              478   
3819                291             447794  ...                8   

3                                                                    \
4    # of Loans Originated $ of Loans Originated # of Disbursements   
0                       33                386770                 35   
1                       40                265784                 40   
2                        8                 88046                  8   
3                      125               1288224                125   
4                        0                     0                  0   
...                    ...                   ...                ...   
3815                   207               1830536                209   
3816                     2                 13300                  2   
3817                     7                 34978                  7   
3818                   490               7567391                522   
3819                     8                 41636                  8   

3                       FFEL GRAD PLUS                        \
4    $ of Disbursements     Recipients # of Loans Originated   
0                192181              5                     5   
1                138160           

### Do the column values appear numeric?
* Check with .dtypes if the columns are integer types

In [49]:
ays_clean2.dtypes

3                  4                    
NaN                OPE ID                   object
                   School                   object
                   State                    object
                   Zip Code                 object
                   School Type              object
FFEL SUBSIDIZED    Recipients                int64
                   # of Loans Originated     int64
                   $ of Loans Originated     int64
                   # of Disbursements        int64
                   $ of Disbursements        int64
FFEL UNSUBSIDIZED  Recipients                int64
                   # of Loans Originated     int64
                   $ of Loans Originated     int64
                   # of Disbursements        int64
                   $ of Disbursements        int64
FFEL PARENT PLUS   Recipients                int64
                   # of Loans Originated     int64
                   $ of Loans Originated     int64
                   # of Disbursements    

### Result: Yes, the column values are numeric (int64)


### Use describe to extract statistics and check for outliers

In [50]:
ays_clean2.describe()

3     FFEL SUBSIDIZED                                              \
4          Recipients # of Loans Originated $ of Loans Originated   
count     3820.000000           3820.000000          3.820000e+03   
mean       735.766230            747.539267          3.404240e+06   
std       2022.902051           2060.049041          9.368663e+06   
min          0.000000              0.000000          0.000000e+00   
25%         18.000000             18.000000          7.229875e+04   
50%        159.000000            160.000000          6.665555e+05   
75%        720.500000            728.250000          3.062910e+06   
max      67590.000000          68099.000000          2.833224e+08   

3                                           FFEL UNSUBSIDIZED  \
4     # of Disbursements $ of Disbursements        Recipients   
count        3820.000000       3.820000e+03       3820.000000   
mean          763.767539       1.708575e+06        713.747382   
std          2116.026539       4.865226e+06       2010.022622   
min             0.000000       0.000000e+00          0.000000   
25%            19.000000       3.585050e+04         15.000000   
50%           166.000000       3.215795e+05        145.000000   
75%           742.000000       1.521889e+06        688.250000   
max         70312.000000       1.548300e+08      62730.000000   

3                                                                     \
4     # of Loans Originated $ of Loans Originated # of Disbursements   
count           3820.000000          3.820000e+03        3820.000000   
mean             762.576440          4.215786e+06         779.637696   
std             2135.076524          1.269787e+07        2192.978381   
min                0.000000          0.000000e+00           0.000000   
25%               15.000000          7.457225e+04          16.000000   
50%              152.000000          6.948935e+05         156.500000   
75%              736.250000          3.454190e+06         757.500000   
max            63556.000000          3.750060e+08       65830.000000   

3                        FFEL PARENT PLUS                        \
4     $ of Disbursements       Recipients # of Loans Originated   
count       3.820000e+03      3820.000000           3820.000000   
mean        2.095755e+06        64.417801             66.702880   
std         6.514066e+06       183.029608            190.040276   
min         0.000000e+00         0.000000              0.000000   
25%         3.663150e+04         0.000000              0.000000   
50%         3.317690e+05         3.000000              3.000000   
75%         1.668416e+06        47.000000             48.000000   
max         1.985650e+08      3272.000000           3339.000000   

3                                                                  \
4     $ of Loans Originated # of Disbursements $ of Disbursements   
count          3.820000e+03        3820.000000       3.820000e+03   
mean           8.415002e+05          68.390052       4.273558e+05   
std            2.846121e+06         193.765012       1.446481e+06   
min            0.000000e+00           0.000000       0.000000e+00   
25%            0.000000e+00           0.000000       0.000000e+00   
50%            2.454050e+04           3.000000       1.179050e+04   
75%            4.573030e+05          48.000000       2.236740e+05   
max            8.802681e+07        3354.000000       4.467873e+07   

3     FFEL GRAD PLUS                                              \
4         Recipients # of Loans Originated $ of Loans Originated   
count    3820.000000           3820.000000          3.820000e+03   
mean       33.791361             36.476963          7.116602e+05   
std       175.446114            190.142882          4.493335e+06   
min         0.000000              0.000000          0.000000e+00   
25%         0.000000              0.000000          0.000000e+00   
50%         0.000000              0.000000          0.000000e+00   
75%         1.000000           

### Result: 0 is in the minimum for all loans. This is likely due to missing values being converted to 0, so we must remove rows that contain many missing values (0s).

## 7. Create a copy of the data and "flatten it"
* Create a copy of this dataframe where layers are "flattened", called <u>ays_clean2_flat</u>
* The flattened dataframe will not have multi-level column names.
* This will aid plotting later on as accessing specific columns will become easier, and we can simply group together the loan type columns by matching the column name to the loan type, such as "SUBSIDIZED."
* Display flat version, <u>ays_clean2_flat</u>
* New columns should contain loan type such as "Recipients_FFEL_SUBSIDIZED".

In [51]:
#Flat version
ays_clean2_flat = ays_clean2.copy()
ays_clean2_flat.columns = [f"{grp}_{col}" for grp, col in ays_clean2.columns]

ays_clean2_flat

,nan_OPE ID,nan_School,nan_State,nan_Zip Code,nan_School Type,FFEL SUBSIDIZED_Recipients,FFEL SUBSIDIZED_# of Loans Originated,FFEL SUBSIDIZED_$ of Loans Originated,FFEL SUBSIDIZED_# of Disbursements,FFEL SUBSIDIZED_$ of Disbursements,...,FFEL PARENT PLUS_Recipients,FFEL PARENT PLUS_# of Loans Originated,FFEL PARENT PLUS_$ of Loans Originated,FFEL PARENT PLUS_# of Disbursements,FFEL PARENT PLUS_$ of Disbursements,FFEL GRAD PLUS_Recipients,FFEL GRAD PLUS_# of Loans Originated,FFEL GRAD PLUS_$ of Loans Originated,FFEL GRAD PLUS_# of Disbursements,FFEL GRAD PLUS_$ of Disbursements
0,00106100,ALASKA PACIFIC UNIVERSITY,AK,995084672,PRIVATE,291,291,1546994,292,830513,...,31,33,386770,35,192181,5,5,69730,5,34865
1,00106300,UNIVERSITY OF ALASKA FAIRBANKS,AK,997757500,PUBLIC,1413,1434,6394735,1455,3290699,...,36,40,265784,40,138160,4,4,68620,4,28988
2,00106500,UNIVERSITY OF ALASKA SOUTHEAST,AK,998018680,PUBLIC,406,409,1866473,439,1044946,...,8,8,88046,8,44024,0,0,0,0,0
3,01146200,UNIVERSITY OF ALASKA ANCHORAGE,AK,995088050,PUBLIC,2998,3042,12780036,3045,6440086,...,123,125,1288224,125,645683,9,11,43878,11,24063
4,02541000,ALASKA CAREER COLLEGE,AK,995071033,PROPRIETARY,38,38,103869,38,52178,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3815,00393200,UNIVERSITY OF WYOMING,WY,820713663,PUBLIC,2699,2735,12341711,2738,6819888,...,195,207,1830536,209,950682,58,65,740877,66,416236
3816,00393300,WESTERN WYOMING COMMUNITY COLLEGE,WY,829010428,PUBLIC,214,214,731883,215,359742,...,2,2,13300,2,6650,0,0,0,0,0
3817,00728900,CENTRAL WYOMING COLLEGE,WY,825012215,PUBLIC,149,154,414959,154,212783,...,7,7,34978,7,17492,0,0,0,0,0
3818,00915700,WYOTECH,WY,820729519,PROPRIETARY,1049,1099,2554580,1220,1498483,...,478,490,7567391,522,4033026,0,0,0,0,0


### Note: The first few columns contain "nan" at the start as they were under the "NaN" header. This must be fixed.


### Fix columns that begin with "nan"
* Remove nan prefix from columns by replacing "nan_" with nothing.
* Display the updated dataframe to check for differences

In [52]:
ays_clean2_flat.columns = [col.replace('nan_', '') for col in ays_clean2_flat.columns]

ays_clean2_flat

,OPE ID,School,State,Zip Code,School Type,FFEL SUBSIDIZED_Recipients,FFEL SUBSIDIZED_# of Loans Originated,FFEL SUBSIDIZED_$ of Loans Originated,FFEL SUBSIDIZED_# of Disbursements,FFEL SUBSIDIZED_$ of Disbursements,...,FFEL PARENT PLUS_Recipients,FFEL PARENT PLUS_# of Loans Originated,FFEL PARENT PLUS_$ of Loans Originated,FFEL PARENT PLUS_# of Disbursements,FFEL PARENT PLUS_$ of Disbursements,FFEL GRAD PLUS_Recipients,FFEL GRAD PLUS_# of Loans Originated,FFEL GRAD PLUS_$ of Loans Originated,FFEL GRAD PLUS_# of Disbursements,FFEL GRAD PLUS_$ of Disbursements
0,00106100,ALASKA PACIFIC UNIVERSITY,AK,995084672,PRIVATE,291,291,1546994,292,830513,...,31,33,386770,35,192181,5,5,69730,5,34865
1,00106300,UNIVERSITY OF ALASKA FAIRBANKS,AK,997757500,PUBLIC,1413,1434,6394735,1455,3290699,...,36,40,265784,40,138160,4,4,68620,4,28988
2,00106500,UNIVERSITY OF ALASKA SOUTHEAST,AK,998018680,PUBLIC,406,409,1866473,439,1044946,...,8,8,88046,8,44024,0,0,0,0,0
3,01146200,UNIVERSITY OF ALASKA ANCHORAGE,AK,995088050,PUBLIC,2998,3042,12780036,3045,6440086,...,123,125,1288224,125,645683,9,11,43878,11,24063
4,02541000,ALASKA CAREER COLLEGE,AK,995071033,PROPRIETARY,38,38,103869,38,52178,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3815,00393200,UNIVERSITY OF WYOMING,WY,820713663,PUBLIC,2699,2735,12341711,2738,6819888,...,195,207,1830536,209,950682,58,65,740877,66,416236
3816,00393300,WESTERN WYOMING COMMUNITY COLLEGE,WY,829010428,PUBLIC,214,214,731883,215,359742,...,2,2,13300,2,6650,0,0,0,0,0
3817,00728900,CENTRAL WYOMING COLLEGE,WY,825012215,PUBLIC,149,154,414959,154,212783,...,7,7,34978,7,17492,0,0,0,0,0
3818,00915700,WYOTECH,WY,820729519,PROPRIETARY,1049,1099,2554580,1220,1498483,...,478,490,7567391,522,4033026,0,0,0,0,0


## 8. Delete rows containing foreign schools as we will only be examining US schools
### Ways to detect:
* "State" column contains value "FC" (Foreign College)
* "School Type" contains value "FOREIGN"
### We will use a command to search for either of the above conditions in order to determine if a school is foreign.
1.  Index the column by matching the column information to either condition above
* Use ~() to negate or perform the opposite of the conditions (include only non-foreign schools)
* Use .str.upper() to force the column name to uppercase in order to match with the string
* Keep only US schools
  
2.  Save to new dataframe, <u>ays_clean3</u>
3.  Display new dataframe to check for foreign schools
4.  Run a command to check for foreign schools


In [53]:
ays_clean3 = ays_clean2_flat[
    ~((ays_clean2_flat['State'].str.upper() == 'FC') | (ays_clean2_flat['State'].str.upper() == 'PR'))
]

ays_clean3

,OPE ID,School,State,Zip Code,School Type,FFEL SUBSIDIZED_Recipients,FFEL SUBSIDIZED_# of Loans Originated,FFEL SUBSIDIZED_$ of Loans Originated,FFEL SUBSIDIZED_# of Disbursements,FFEL SUBSIDIZED_$ of Disbursements,...,FFEL PARENT PLUS_Recipients,FFEL PARENT PLUS_# of Loans Originated,FFEL PARENT PLUS_$ of Loans Originated,FFEL PARENT PLUS_# of Disbursements,FFEL PARENT PLUS_$ of Disbursements,FFEL GRAD PLUS_Recipients,FFEL GRAD PLUS_# of Loans Originated,FFEL GRAD PLUS_$ of Loans Originated,FFEL GRAD PLUS_# of Disbursements,FFEL GRAD PLUS_$ of Disbursements
0,00106100,ALASKA PACIFIC UNIVERSITY,AK,995084672,PRIVATE,291,291,1546994,292,830513,...,31,33,386770,35,192181,5,5,69730,5,34865
1,00106300,UNIVERSITY OF ALASKA FAIRBANKS,AK,997757500,PUBLIC,1413,1434,6394735,1455,3290699,...,36,40,265784,40,138160,4,4,68620,4,28988
2,00106500,UNIVERSITY OF ALASKA SOUTHEAST,AK,998018680,PUBLIC,406,409,1866473,439,1044946,...,8,8,88046,8,44024,0,0,0,0,0
3,01146200,UNIVERSITY OF ALASKA ANCHORAGE,AK,995088050,PUBLIC,2998,3042,12780036,3045,6440086,...,123,125,1288224,125,645683,9,11,43878,11,24063
4,02541000,ALASKA CAREER COLLEGE,AK,995071033,PROPRIETARY,38,38,103869,38,52178,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3815,00393200,UNIVERSITY OF WYOMING,WY,820713663,PUBLIC,2699,2735,12341711,2738,6819888,...,195,207,1830536,209,950682,58,65,740877,66,416236
3816,00393300,WESTERN WYOMING COMMUNITY COLLEGE,WY,829010428,PUBLIC,214,214,731883,215,359742,...,2,2,13300,2,6650,0,0,0,0,0
3817,00728900,CENTRAL WYOMING COLLEGE,WY,825012215,PUBLIC,149,154,414959,154,212783,...,7,7,34978,7,17492,0,0,0,0,0
3818,00915700,WYOTECH,WY,820729519,PROPRIETARY,1049,1099,2554580,1220,1498483,...,478,490,7567391,522,4033026,0,0,0,0,0


### Observation: The first few columns have proper names instead of leading with "nan_"
### Check for foreign schools

In [54]:
#Check for FC or PR in States
print("Unique states:", ays_clean3['State'].unique())

Unique states: ['AK' 'AL' 'AR' 'AZ' 'CA' 'CO' 'CT' 'DC' 'DE' 'FL' 'GA' 'GU' 'HI' 'IA'
 'ID' 'IL' 'IN' 'KS' 'KY' 'LA' 'MA' 'MD' 'ME' 'MI' 'MN' 'MO' 'MS' 'MT'
 'NC' 'ND' 'NE' 'NH' 'NJ' 'NM' 'NR' 'NV' 'NY' 'OH' 'OK' 'OR' 'PA' 'RI'
 'SC' 'SD' 'TN' 'TX' 'UT' 'VA' 'VT' 'WA' 'WI' 'WV' 'WY']


### Result: No 'FC' or 'PR' detected, no foreign schools.

## 9. Delete rows containing all missing values, which are marked with "0"
* Missing values have been replaced with "0", so it is difficult to discern true 0 values from missing values
* For this reason, we will only be removing rows that contain multiple zeroes as they are true outliers to the data
### Cleaning process:
1. Identify Numeric Columns (from column 5 onward), so that we only alter numbers
2. Count Zeroes in Each Row (only in numeric columns)
3. Drop rows containing 2 or more zeroes, as this is a strong indicator of missing values (due to missing values being converted to 0)
* Reset index to discard old index instead of adding it as a column
* Save this to a new dataframe: <u>ays_clean4</u>

4. Check the changes by counting the amount of rows in the new data set. Count the amount of dropped rows by finding the amount of rows containing 2 or more zeroes.
* Use len(dataframe) to determine number of remaining rows
* Use len(zero_counts[zero+counts>1]) to determine number of dropped rows

5. Display new data, <u>ays_clean4</u>


In [55]:
numeric_cols = ays_clean3.columns[5:]
zero_counts = (ays_clean3[numeric_cols] == 0).sum(axis=1)
ays_clean4 = ays_clean3[zero_counts <= 1].reset_index(drop=True)
print(f"Remaining rows: {len(ays_clean4)}")
print(f"Dropped rows with >1 zero: {len(zero_counts[zero_counts > 1])}")


Remaining rows: 564
Dropped rows with >1 zero: 2850


## Display cleaned data and descriptive statistics
* Display dataframe <u>ays_clean4</u>
* Use .describe() to extract descriptive statistics

In [56]:
ays_clean4

,OPE ID,School,State,Zip Code,School Type,FFEL SUBSIDIZED_Recipients,FFEL SUBSIDIZED_# of Loans Originated,FFEL SUBSIDIZED_$ of Loans Originated,FFEL SUBSIDIZED_# of Disbursements,FFEL SUBSIDIZED_$ of Disbursements,...,FFEL PARENT PLUS_Recipients,FFEL PARENT PLUS_# of Loans Originated,FFEL PARENT PLUS_$ of Loans Originated,FFEL PARENT PLUS_# of Disbursements,FFEL PARENT PLUS_$ of Disbursements,FFEL GRAD PLUS_Recipients,FFEL GRAD PLUS_# of Loans Originated,FFEL GRAD PLUS_$ of Loans Originated,FFEL GRAD PLUS_# of Disbursements,FFEL GRAD PLUS_$ of Disbursements
0,00106100,ALASKA PACIFIC UNIVERSITY,AK,995084672,PRIVATE,291,291,1546994,292,830513,...,31,33,386770,35,192181,5,5,69730,5,34865
1,00106300,UNIVERSITY OF ALASKA FAIRBANKS,AK,997757500,PUBLIC,1413,1434,6394735,1455,3290699,...,36,40,265784,40,138160,4,4,68620,4,28988
2,01146200,UNIVERSITY OF ALASKA ANCHORAGE,AK,995088050,PUBLIC,2998,3042,12780036,3045,6440086,...,123,125,1288224,125,645683,9,11,43878,11,24063
3,00100200,ALABAMA AGRICULTURAL & MECHANICAL UNIVERSITY,AL,357621357,PUBLIC,2675,2692,12343721,2701,6178188,...,231,236,1744374,239,927673,2,2,9524,2,6262
4,00100300,FAULKNER UNIVERSITY,AL,361093398,PRIVATE,1392,1403,6244282,1426,3470561,...,78,78,611850,79,347361,188,203,4204847,208,2143612
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
559,00380600,ALDERSON BROADDUS COLLEGE,WV,264161051,PRIVATE,322,327,1430393,327,724089,...,43,46,362372,46,186638,17,19,189386,19,107549
560,00380700,MOUNTAIN STATE UNIVERSITY,WV,258015624,PRIVATE,1285,1288,3299331,1400,1812988,...,6,6,30348,6,15175,59,59,440493,61,228951
561,00381800,UNIVERSITY OF CHARLESTON,WV,253041099,PRIVATE,534,544,3220394,551,1658291,...,19,21,174804,21,87402,100,107,1502249,107,757897
562,00383100,WHEELING JESUIT UNIVERSITY,WV,260036243,PRIVATE,642,646,2841866,648,1519125,...,79,82,816500,83,421514,25,25,265555,25,163898


In [57]:
ays_clean4.describe()

,FFEL SUBSIDIZED_Recipients,FFEL SUBSIDIZED_# of Loans Originated,FFEL SUBSIDIZED_$ of Loans Originated,FFEL SUBSIDIZED_# of Disbursements,FFEL SUBSIDIZED_$ of Disbursements,FFEL UNSUBSIDIZED_Recipients,FFEL UNSUBSIDIZED_# of Loans Originated,FFEL UNSUBSIDIZED_$ of Loans Originated,FFEL UNSUBSIDIZED_# of Disbursements,FFEL UNSUBSIDIZED_$ of Disbursements,FFEL PARENT PLUS_Recipients,FFEL PARENT PLUS_# of Loans Originated,FFEL PARENT PLUS_$ of Loans Originated,FFEL PARENT PLUS_# of Disbursements,FFEL PARENT PLUS_$ of Disbursements,FFEL GRAD PLUS_Recipients,FFEL GRAD PLUS_# of Loans Originated,FFEL GRAD PLUS_$ of Loans Originated,FFEL GRAD PLUS_# of Disbursements,FFEL GRAD PLUS_$ of Disbursements
count,564.000000,564.000000,5.640000e+02,564.000000,5.640000e+02,564.000000,564.000000,5.640000e+02,564.000000,5.640000e+02,564.000000,564.000000,5.640000e+02,564.000000,5.640000e+02,564.000000,564.000000,5.640000e+02,564.000000,5.640000e+02
mean,2685.127660,2731.985816,1.354500e+07,2768.578014,6.873014e+06,2695.831560,2866.721631,1.714124e+07,2905.969858,8.647460e+06,273.448582,283.728723,3.801764e+06,287.239362,1.938498e+06,158.820922,171.753546,3.312007e+06,174.613475,1.637070e+06
std,3965.147873,4031.637495,1.850699e+07,4140.310898,9.746493e+06,3821.443153,4005.704453,2.469103e+07,4120.607471,1.293096e+07,366.860894,380.938813,6.143853e+06,384.182319,3.136238e+06,383.643093,417.050416,1.018857e+07,423.806492,5.044708e+06
min,1.000000,1.000000,4.315000e+03,1.000000,3.440000e+03,1.000000,1.000000,1.200000e+04,1.000000,6.000000e+03,1.000000,1.000000,6.310000e+03,1.000000,3.155000e+03,1.000000,1.000000,3.410000e+02,1.000000,2.900000e+02
25%,750.500000,757.750000,3.941575e+06,764.000000,1.987668e+06,758.750000,808.000000,4.563331e+06,811.750000,2.285558e+06,60.750000,62.750000,6.366578e+05,64.000000,3.474038e+05,3.000000,4.000000,2.808350e+04,4.000000,1.418275e+04
50%,1521.000000,1532.000000,7.771066e+06,1569.500000,3.901344e+06,1575.000000,1660.000000,9.101471e+06,1700.500000,4.466388e+06,164.000000,169.000000,1.901662e+06,169.500000,9.787745e+05,18.000000,19.000000,2.086165e+05,20.000000,1.070475e+05
75%,3288.250000,3313.750000,1.667718e+07,3357.500000,8.400126e+06,3346.250000,3487.500000,2.145915e+07,3516.250000,1.055839e+07,336.000000,353.250000,4.745630e+06,355.750000,2.363046e+06,104.750000,111.000000,1.429667e+06,114.750000,7.100732e+05
max,67590.000000,68099.000000,2.833224e+08,70312.000000,1.548300e+08,62730.000000,63556.000000,3.750060e+08,65830.000000,1.985650e+08,3272.000000,3339.000000,8.802681e+07,3354.000000,4.467873e+07,4112.000000,4286.000000,1.311844e+08,4323.000000,6.673883e+07


### Result: minimum is no longer 0. The data no longer contains outliers or foreign schools. The data is now cleaned

## Data Cleaning: Copy cleaned data to new variable
* Copy the cleaned data to a new variable, <u>ays_cleaned</u>
* Display the new variable to check that it is the same
* We are saving this to a clearly defined variable so that we can differentiate this final cleaned dataframe from the uncleaned copies. This copy will be exported to a CSV file at the end of this file so that we may import it into the next project

In [58]:
ays_cleaned = ays_clean4.copy()

In [59]:
ays_cleaned

,OPE ID,School,State,Zip Code,School Type,FFEL SUBSIDIZED_Recipients,FFEL SUBSIDIZED_# of Loans Originated,FFEL SUBSIDIZED_$ of Loans Originated,FFEL SUBSIDIZED_# of Disbursements,FFEL SUBSIDIZED_$ of Disbursements,...,FFEL PARENT PLUS_Recipients,FFEL PARENT PLUS_# of Loans Originated,FFEL PARENT PLUS_$ of Loans Originated,FFEL PARENT PLUS_# of Disbursements,FFEL PARENT PLUS_$ of Disbursements,FFEL GRAD PLUS_Recipients,FFEL GRAD PLUS_# of Loans Originated,FFEL GRAD PLUS_$ of Loans Originated,FFEL GRAD PLUS_# of Disbursements,FFEL GRAD PLUS_$ of Disbursements
0,00106100,ALASKA PACIFIC UNIVERSITY,AK,995084672,PRIVATE,291,291,1546994,292,830513,...,31,33,386770,35,192181,5,5,69730,5,34865
1,00106300,UNIVERSITY OF ALASKA FAIRBANKS,AK,997757500,PUBLIC,1413,1434,6394735,1455,3290699,...,36,40,265784,40,138160,4,4,68620,4,28988
2,01146200,UNIVERSITY OF ALASKA ANCHORAGE,AK,995088050,PUBLIC,2998,3042,12780036,3045,6440086,...,123,125,1288224,125,645683,9,11,43878,11,24063
3,00100200,ALABAMA AGRICULTURAL & MECHANICAL UNIVERSITY,AL,357621357,PUBLIC,2675,2692,12343721,2701,6178188,...,231,236,1744374,239,927673,2,2,9524,2,6262
4,00100300,FAULKNER UNIVERSITY,AL,361093398,PRIVATE,1392,1403,6244282,1426,3470561,...,78,78,611850,79,347361,188,203,4204847,208,2143612
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
559,00380600,ALDERSON BROADDUS COLLEGE,WV,264161051,PRIVATE,322,327,1430393,327,724089,...,43,46,362372,46,186638,17,19,189386,19,107549
560,00380700,MOUNTAIN STATE UNIVERSITY,WV,258015624,PRIVATE,1285,1288,3299331,1400,1812988,...,6,6,30348,6,15175,59,59,440493,61,228951
561,00381800,UNIVERSITY OF CHARLESTON,WV,253041099,PRIVATE,534,544,3220394,551,1658291,...,19,21,174804,21,87402,100,107,1502249,107,757897
562,00383100,WHEELING JESUIT UNIVERSITY,WV,260036243,PRIVATE,642,646,2841866,648,1519125,...,79,82,816500,83,421514,25,25,265555,25,163898


# <u>Data Cleaning: FSL</u>
1. Inspect entire dataset
2. Inspect random samples of dataset

In [60]:
fsl_clean

,SchoolCode,SchoolName,Address,City,StateCode,ZipCode,Province,Country,PostalCode
0,B04724,WIDENER UNIV SCHOOL OF LAW - DE,4601 CONCORD PIKE/PO BOX 7474,WILMINGTON,DE,19803,NaN,NaN,NaN
1,B06171,CENTER FOR ADVANCED STUDIES OF PUER,BOX S-4467,SAN JUAN,PR,902,NaN,NaN,NaN
2,B06511,PENTECOSTAL THEOLOGICAL SEMINARY,PO BOX 3330,CLEVELAND,TN,37320,NaN,NaN,NaN
3,B07022,THE CHICAGO SCHOOL OF PROF PSYCHOLOGY,325 NORTH WELLS STREET,CHICAGO,IL,60610,NaN,NaN,NaN
4,B07624,NATIONAL COLLEGE OF NATURAL MEDICINE,049 SW PORTER,PORTLAND,OR,97201,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
6975,042517,HOPE COLLEGE OF ARTS AND SCIENCES,1200 SOUTH WEST 3RD STREET,POMPANO BEACH,FL,33069,NaN,NaN,NaN
6976,E40419,BEAUTY INSTITUTE SCHWARZKOPF PROFESSIONA,1411 RAILROAD AVENUE,BELLINGHAM,WA,98225,NaN,NaN,NaN
6977,042205,BUTTE COUNTY REGIONAL OCCUPATIONAL PROGR,2491 CARMICHAEL DRIVE,CHICO,CA,95928,NaN,NaN,NaN
6978,G42404,UNIVERSIDAD ANA G. MENDEZ - CAMPUS VIRTU,1552 AVENUE PONCE DE LEON,SAN JUAN,PR,926,NaN,NaN,NaN


### Observations:
* Proper column names observed, although in title case with no whitespace. This may be changed.
* Contains foreign schools (PR and FC in StateCode)
* Contains extra columns exclusively for foreign schools: ZipCode, Province, Country, and PostalCode
### Data Cleaning Plan:
1. Change column names to include whitespace where available
2. Remove foreign schools (containing StateCode FC or PR)
3. Remove final 3 columns as they only pertain to foreign schools and contain missing values for US-only schools (Province, Country, PostalCode). Can be done with slicing.
4. Inspect the new dataframe

## 1. Change column names to include whitespace where available
* Use a regular expression (regex) to locate a capital letter that is not at the start of the string and insert a space before it.
* Use re.sub to substitute the capital letter with a space
* (?<!^) = not at the start of the string
* (?=[A-Z]) = right before an uppercase letter
* Assign these new names to the columns in fsl_clean
* Display the dataframe and check for proper column spaces
* Use .columns to manually view column names and check for proper spaces

In [61]:
fsl_clean.columns = [re.sub(r'(?<!^)(?=[A-Z])', ' ', col) for col in fsl_clean.columns]

In [62]:
fsl_clean

,School Code,School Name,Address,City,State Code,Zip Code,Province,Country,Postal Code
0,B04724,WIDENER UNIV SCHOOL OF LAW - DE,4601 CONCORD PIKE/PO BOX 7474,WILMINGTON,DE,19803,NaN,NaN,NaN
1,B06171,CENTER FOR ADVANCED STUDIES OF PUER,BOX S-4467,SAN JUAN,PR,902,NaN,NaN,NaN
2,B06511,PENTECOSTAL THEOLOGICAL SEMINARY,PO BOX 3330,CLEVELAND,TN,37320,NaN,NaN,NaN
3,B07022,THE CHICAGO SCHOOL OF PROF PSYCHOLOGY,325 NORTH WELLS STREET,CHICAGO,IL,60610,NaN,NaN,NaN
4,B07624,NATIONAL COLLEGE OF NATURAL MEDICINE,049 SW PORTER,PORTLAND,OR,97201,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
6975,042517,HOPE COLLEGE OF ARTS AND SCIENCES,1200 SOUTH WEST 3RD STREET,POMPANO BEACH,FL,33069,NaN,NaN,NaN
6976,E40419,BEAUTY INSTITUTE SCHWARZKOPF PROFESSIONA,1411 RAILROAD AVENUE,BELLINGHAM,WA,98225,NaN,NaN,NaN
6977,042205,BUTTE COUNTY REGIONAL OCCUPATIONAL PROGR,2491 CARMICHAEL DRIVE,CHICO,CA,95928,NaN,NaN,NaN
6978,G42404,UNIVERSIDAD ANA G. MENDEZ - CAMPUS VIRTU,1552 AVENUE PONCE DE LEON,SAN JUAN,PR,926,NaN,NaN,NaN


In [63]:
fsl_clean.columns

Index(['School Code', 'School Name', 'Address', 'City', 'State Code',
       'Zip Code', 'Province', 'Country', 'Postal Code'],
      dtype='object')

### Success: Column names now contain spaces where appropriate

## 2. Delete rows containing foreign schools as we will only be examining US schools
### Ways to detect:
* "State" column contains value "FC" (Foreign College)
* "School Type" contains value "FOREIGN"
### We will use a command to search for either of the above conditions in order to determine if a school is foreign.
1.  Index the column by matching the column information to either condition above
* Use ~() to negate or perform the opposite of the conditions (include only non-foreign schools)
* Use .str.upper() to force the column name to uppercase in order to match with the string
* Keep only US schools
  
2.  Save to new dataframe, <u>fsl_clean2</u>
3.  Display new dataframe to check for foreign schools
4.  Run a command to check for foreign schools

In [64]:
fsl_clean2 = fsl_clean[
    ~((fsl_clean['State Code'].str.upper() == 'FC') | (fsl_clean['State Code'].str.upper() == 'PR'))
]

In [65]:
fsl_clean2

,School Code,School Name,Address,City,State Code,Zip Code,Province,Country,Postal Code
0,B04724,WIDENER UNIV SCHOOL OF LAW - DE,4601 CONCORD PIKE/PO BOX 7474,WILMINGTON,DE,19803,NaN,NaN,NaN
2,B06511,PENTECOSTAL THEOLOGICAL SEMINARY,PO BOX 3330,CLEVELAND,TN,37320,NaN,NaN,NaN
3,B07022,THE CHICAGO SCHOOL OF PROF PSYCHOLOGY,325 NORTH WELLS STREET,CHICAGO,IL,60610,NaN,NaN,NaN
4,B07624,NATIONAL COLLEGE OF NATURAL MEDICINE,049 SW PORTER,PORTLAND,OR,97201,NaN,NaN,NaN
5,B07625,OREGON COL OF ORIENTAL MEDICINE,10525 SE CHERRY BLOSSOM DR,PORTLAND,OR,97216,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
6974,042503,HAIR ACADEMY SCHOOL OF BARBERING & BEAUT,1013 SOUTH COLLEGE AVENUE,NEWARK,DE,19713,NaN,NaN,NaN
6975,042517,HOPE COLLEGE OF ARTS AND SCIENCES,1200 SOUTH WEST 3RD STREET,POMPANO BEACH,FL,33069,NaN,NaN,NaN
6976,E40419,BEAUTY INSTITUTE SCHWARZKOPF PROFESSIONA,1411 RAILROAD AVENUE,BELLINGHAM,WA,98225,NaN,NaN,NaN
6977,042205,BUTTE COUNTY REGIONAL OCCUPATIONAL PROGR,2491 CARMICHAEL DRIVE,CHICO,CA,95928,NaN,NaN,NaN


### Check for foreign schools (FC or PR in State Code)
* We will use booleans to check if FC or PR exist as a value in the State Code column.
* If it returns true, then there are still foreign schools in this dataset.
* If it returns false, we have successfully removed all foreign schools.

In [66]:
'FC' in fsl_clean2['State Code'].values  #True if FC exists

False

In [67]:
'PR' in fsl_clean2['State Code'].values  #True if PR exists

False

### Observation: Both statements returned false, so there are no foreign schools left in the data.

## 3. Remove final 3 columns as they only pertain to foreign schools and contain missing values for US-only schools (Province, Country, Postal Code). Can be done with slicing.
* Use .iloc[:, :-3] to keep all rows and all columns except the last 3 (exclude the last 3)
* This drops the columns at indexpositions -3, -2, and -1: Province, Country, and Postal Code.
* These columns are not needed since they are only used for foreign schools and we will only explore US schools. Missing values are unneeded.
* Save this to a new dataframe, <u>fsl_clean3</u>
* View the new dataframe to confirm changes

In [68]:
fsl_clean3 = fsl_clean.iloc[:, :-3]


In [69]:
fsl_clean3

,School Code,School Name,Address,City,State Code,Zip Code
0,B04724,WIDENER UNIV SCHOOL OF LAW - DE,4601 CONCORD PIKE/PO BOX 7474,WILMINGTON,DE,19803
1,B06171,CENTER FOR ADVANCED STUDIES OF PUER,BOX S-4467,SAN JUAN,PR,902
2,B06511,PENTECOSTAL THEOLOGICAL SEMINARY,PO BOX 3330,CLEVELAND,TN,37320
3,B07022,THE CHICAGO SCHOOL OF PROF PSYCHOLOGY,325 NORTH WELLS STREET,CHICAGO,IL,60610
4,B07624,NATIONAL COLLEGE OF NATURAL MEDICINE,049 SW PORTER,PORTLAND,OR,97201
...,...,...,...,...,...,...
6975,042517,HOPE COLLEGE OF ARTS AND SCIENCES,1200 SOUTH WEST 3RD STREET,POMPANO BEACH,FL,33069
6976,E40419,BEAUTY INSTITUTE SCHWARZKOPF PROFESSIONA,1411 RAILROAD AVENUE,BELLINGHAM,WA,98225
6977,042205,BUTTE COUNTY REGIONAL OCCUPATIONAL PROGR,2491 CARMICHAEL DRIVE,CHICO,CA,95928
6978,G42404,UNIVERSIDAD ANA G. MENDEZ - CAMPUS VIRTU,1552 AVENUE PONCE DE LEON,SAN JUAN,PR,926


### Success: The last 3 columns have been removed entirely
* The data now contains no foreign schools and no missing columns.
* This dataframe does not contain any loan information, so we do not need to remove any rows containing 0s.
* Data cleaned successfully.

## Data Cleaning: Copy cleaned data to new variable
* Copy the cleaned data to a new variable, <u>fsl_cleaned</u>
* Display the new variable to check that it is the same
* We are saving this to a clearly defined variable so that we can differentiate this final cleaned dataframe from the uncleaned copies. This copy will be exported to a CSV file at the end of this file so that we may import it into the next project

In [70]:
fsl_cleaned = fsl_clean3.copy()

In [71]:
fsl_cleaned

,School Code,School Name,Address,City,State Code,Zip Code
0,B04724,WIDENER UNIV SCHOOL OF LAW - DE,4601 CONCORD PIKE/PO BOX 7474,WILMINGTON,DE,19803
1,B06171,CENTER FOR ADVANCED STUDIES OF PUER,BOX S-4467,SAN JUAN,PR,902
2,B06511,PENTECOSTAL THEOLOGICAL SEMINARY,PO BOX 3330,CLEVELAND,TN,37320
3,B07022,THE CHICAGO SCHOOL OF PROF PSYCHOLOGY,325 NORTH WELLS STREET,CHICAGO,IL,60610
4,B07624,NATIONAL COLLEGE OF NATURAL MEDICINE,049 SW PORTER,PORTLAND,OR,97201
...,...,...,...,...,...,...
6975,042517,HOPE COLLEGE OF ARTS AND SCIENCES,1200 SOUTH WEST 3RD STREET,POMPANO BEACH,FL,33069
6976,E40419,BEAUTY INSTITUTE SCHWARZKOPF PROFESSIONA,1411 RAILROAD AVENUE,BELLINGHAM,WA,98225
6977,042205,BUTTE COUNTY REGIONAL OCCUPATIONAL PROGR,2491 CARMICHAEL DRIVE,CHICO,CA,95928
6978,G42404,UNIVERSIDAD ANA G. MENDEZ - CAMPUS VIRTU,1552 AVENUE PONCE DE LEON,SAN JUAN,PR,926


# <u>Data Cleaning: Export Clean Data</u>
* Save each cleaned dataframe to the same library containing this notebook
* In the next project, we will be able to import these files and continue data exploration.

In [73]:
qa_cleaned.to_csv("qaclean.csv", index="false")

In [74]:
ays_cleaned.to_csv("aysclean.csv", index="false")

In [75]:
fsl_cleaned.to_csv("fslclean.csv", index="false")


In [20]:
qa_clean4.to_csv("qacleaning.csv", index="false")

### Success: After running the above 3 lines of code, the following files should appear in the same directory as this notebook:
* qaclean.csv
* aysclean.csv
* fslclean.csv
* qacleaning.csv (Used in Milestone 4 as qaclean.csv is not cleaned enough.)